In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce

In [2]:
#id_uf + sigla_uf
id_uf = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/id_uf.csv', dtype='string')
id_uf.rename(columns={'sigla':'sigla_uf'}, inplace=True)

#id_uf + sigla_uf + id_regiao
id_reg = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/id_reg.csv', dtype='string')
id_reg['regiao'] = id_reg['regiao'].replace(['Norte','Nordeste','Sudeste', 'Sul', 'Centro-Oeste'],['1', '2','3','4','5'])
id_reg.rename(columns={'regiao':'id_regiao'}, inplace=True)

anos = [1995, 1997, 1999, 2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]
ufs = id_uf['sigla_uf'].tolist()

In [3]:
#output particao
for ano in anos:
  for uf in ufs:
    directory = '/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano={}/sigla_uf={}'.format(ano, uf)
    if not os.path.exists(directory):
      os.makedirs(directory)

#temp particao
for ano in anos:
  for uf in ufs:
    directory = '/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano={}/sigla_uf={}'.format(ano, uf)
    if not os.path.exists(directory):
      os.makedirs(directory)

In [3]:
ordem = [
            'ano', 'id_regiao', 'sigla_uf', 'id_municipio', 'area', 'mascara', 'ano_mascara',
            'id_escola', 'rede', 'localizacao', 'id_turma', 'turno', 'serie', 'id_aluno', 'situacao_censo',
            'disciplina', 'preenchimento_caderno', 'presenca', 'caderno', 'bloco', 'bloco_1', 'bloco_2', 'bloco_3',
            'bloco_aberto_1', 'bloco_aberto_2', 'item_bloco', 'respostas', 'respostas_bloco_1', 'respostas_bloco_2',
            'respostas_bloco_3', 'conceito_q1', 'conceito_q2', 'gabarito', 'gabarito_bloco_1', 'gabarito_bloco_2',
            'gabarito_bloco_3', 'indicador_proficiencia', 'indicador_prova_brasil', 'amostra', 'estrato', 'estrato_ciencias', 'peso_escola',
            'peso_turma', 'peso_aluno_turma', 'peso_aluno', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
            'erro_padrao_saeb', 'desempenho_aluno', 'preenchimento_questionario', 'sexo', 'raca_cor', 'ano_nascimento',
            'mes_nascimento', 'idade', 'faixa_etaria', 'estado_civil', 'possui_filhos', 'com_quem_mora', 'mora_mae',
            'mora_pai', 'mora_irmaos', 'mora_avos', 'mora_conjugue', 'mora_filhos', 'mora_outros_parentes', 'mora_amigos',
            'idioma_domicilio', 'quantidade_pessoas_domicilio','quantidade_pessoas_despesas_domicilio', 'situacao_economica', 'possui_trabalho',
            'turma_bolsa_escola', 'aluno_bolsa_escola', 'possui_moradia_rua_urbanizada', 'possui_agua_encanada',
            'possui_eletrecidade', 'possui_eletrodomestico_sem_eletricidade', 'possui_casa_dormitorio',
            'possui_casa_quarto_individual', 'possui_casa_cozinha', 'possui_casa_banheiro', 'possui_casa_sala', 'possui_automovel',
            'possui_casa_garagem', 'possui_geladeira', 'possui_geladeira_freezer',
            'possui_freezer', 'possui_microondas', 'possui_maquina_lavar_roupa', 'possui_aspirador_po', 'possui_radio',
            'possui_tv', 'possui_tv_assinatura', 'possui_videocassete_dvd', 'possui_internet', 'possui_computador',
            'possui_computador_sem_internet', 'possui_tablet', 'possui_telefone', 'possui_celular', 'possui_escrivaninha',
            'possui_enciclopedia', 'possui_atlas', 'possui_dicionario', 'possui_calculadora',
            'possui_diarista', 'possui_empregada_domestica', 'possui_empregada_domestica_cinco_dias', 
            'escolaridade_mae', 'mae_sabe_ler_escrever', 'mae_le', 'ocupacao_mae', 'escolaridade_pai',
            'pai_sabe_ler_escrever', 'pai_le', 'ocupacao_pai', 'pessoa_acompanha_vida_escolar', 'escolaridade_pessoa',
            'responsaveis_conhecem_diretor', 'responsaveis_conhecem_professor', 'responsaveis_conversam_professor_diretor',
            'frequencia_responsaveis_conversam_diretor', 'frequencia_responsaveis_conversam_professor',
            'responsaveis_conhecem_amigo', 'responsaveis_conhecem_responsavel_amigo', 'responsaveis_almocam_jantam_contigo',
            'responsaveis_ouvem_musica_contigo', 'responsaveis_conversam_livros_contigo', 'responsaveis_conversam_filmes_contigo',
            'responsaveis_conversam_programas_tv_contigo', 'responsaveis_conversam_amigos', 'responsaveis_conversam_amigos_escola',
            'responsaveis_conversam_responsaveis_amigo', 'responsaveis_conversam_outros_assuntos_contigo',
            'responsaveis_conversam_escola', 'responsaveis_conversam_boletim', 'responsaveis_conversam_comportamento',
            'responsaveis_cobram_realizacao_licao_casa', 'responsaveis_ajudam_realizacao_licao_casa', 
            'responsaveis_incentivam_estudos', 'responsaveis_incentivam_leitura', 'responsaveis_incentivam_realizacao_licao_casa',
            'responsaveis_incentivam_comparecer_aulas', 'responsaveis_incentivam_pontualidade', 'responsaveis_incentivam_boas_notas',
            'responsaveis_comparecem_reuniao_pais', 'responsaveis_participam_festas_escola', 
            'responsaveis_participam_trabalho_voluntario', 'tempo_chegada_escola', 'forma_chegada_escola', 'inicio_estudos', 
            'quantos_anos_primeiro_grau', 'quantos_anos_segundo_grau', 'idade_entrada_escola', 'quantidade_mudancas_escola',
            'presenca_professor', 'supletivo_ef', 'rede_ef', 'reprovacao', 'evasao_escolar_ate_final_ano',
            'evasao_escolar_volta_antes_final_ano', 'evasao_escolar_temporaria', 'motivo_evasao_escolar', 'faltas_aula',
            'motivo_faltas', 'desempenho_prejudicado_falta', 'tempo_lazer', 'tipo_programacao_tv', 'tempo_cursos',
            'tempo_trabalho_domestico', 'tempo_estudos', 
            'quantidade_livros', 'recebe_jornais', 'recebe_revistas', 'leitura_jornais', 'leitura_noticias', 
            'leitura_livros_geral', 'leitura_literatura', 'leitura_literatura_infantojuvenil', 'leitura_historia_quadrinhos',
            'leitura_revistas_tematicas', 'leitura_revista_comportamento', 'leitura_revistas_geral', 'leitura_internet',
            'leitura_outros_materiais', 'frequenta_biblioteca', 'frequenta_cinema', 'frequenta_espetaculo_exposicao',
            'frequenta_museu', 'frequenta_teatro', 'frequenta_concerto', 'frequenta_festas_comunidade', 'participa_gremio_escolar',
            'participa_associacao_desportiva', 'participa_associacao_moradores', 'participa_sindicato',
            'participa_movimento_religioso', 'participa_partido_politico', 'participa_ong', 'nao_participa', 
            'participa_atividades_esporte', 'participa_atividades_artisticas', 'participa_trabalho_solidario',
            'participa_reforco_escolar', 'participa_excursao_acampamento', 'participa_festas', 'curso_atividades_artisticas',
            'curso_reforco_escolar', 'curso_idioma_externo', 'curso_informatica', 'curso_outros', 'correcao_licao_casa_alunos',
            'quando_nao_entende_materia', 'gosta_estudar_disciplina', 'entendimento_ensino', 'disciplina_auxilia_compreensao',
            'professor_disciplina', 'faz_licao_casa', 'tempo_licao_casa', 'professor_leitura_livros_licao_casa',
            'correcao_licao_casa', 'consulta_jornais_revistas_licao_casa', 'utiliza_computador_licao_casa', 'utiliza_computador',
            'utiliza_computador_escola', 'utiliza_biblioteca_escola', 'utiliza_biblioteca_externa', 'existe_lugar_calmo_estudos',
            'comparacao_colegas_disciplina', 'boas_notas_disciplina', 'fez_redacao', 'desempenho_disciplina', 'professor_importa_voce_diz',
            'professor_conversa_avaliacao', 'professor_elogia_merecimento','professor_ajuda_se_necessario', 
            'professor_atende_prontamente', 'professor_trata_carinho', 'professor_esforca_aprendizado', 
            'professor_utiliza_espacos_externos', 'professor_espera_silencio', 'professor_cobra', 'brigou_professor',
            'turma_exclui', 'familia_avisada_falta', 'alunos_desordeiros', 'alunos_atentos', 'alunos_obedientes',
            'possui_amigos_sala', 'estuda_habitualmente', 'estuda_fazendo_licao_casa', 'estuda_fazendo_resumos',
            'estuda_lendo_relendo_livros', 'estuda_refazendo_exercicios', 'estuda_fazendo_anotacoes', 'estuda_lendo_anotacoes',
            'estuda_somente_assistindo_aulas', 'estuda_fora_sala_aula', 'motivo_estar_escola', 'utiliza_aprendizado_diariamente',
            'pretensao_futura', 'opiniao_teste', 'opiniao_frase_1', 'opiniao_frase_2', 'opiniao_frase_3'
]

#cria df para completar todos os anos
add_df = pd.DataFrame(columns=ordem)

##### **1995**

In [4]:
mt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb95_aluno_matematica_8serie.csv', sep=';', encoding='latin-1') #prova matematica
pt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb95_aluno_portugues_8serie.csv', sep=';', encoding='latin-1') #prova portugues
df1 = pd.concat([mt_8s, pt_8s], ignore_index=True, sort=True)
df = mt_8s.append([pt_8s], ignore_index=True)
del mt_8s, pt_8s
df['PROFIC'] = df['PROFIC'].apply(lambda x: x.replace(',','.')).astype('float64')

#desmembra data_nascimento em ano e mês, além de criar variável idade
df['mes_nascimento'] = df['A081_001'].str[2:4].replace('', np.nan).fillna(0)
df['mes_nascimento'] = df['mes_nascimento'].replace('  ', np.nan).fillna(0)
df['mes_nascimento'] = pd.to_numeric(df['mes_nascimento']).astype('int64')
df['mes_nascimento'] = df['mes_nascimento'].replace(0, '')
df['ano_nascimento'] = '19' + df['A081_001'].str[4:6].replace('', np.nan)
df['ano_nascimento'] = df['ano_nascimento'].replace('19',np.nan).fillna(0)
df['ano_nascimento'] = pd.to_numeric(df['ano_nascimento']).astype('int64')
df['ano_nascimento'] = df['ano_nascimento'].replace(0, '')

rename95 = {'ï»¿ANO':'ano', 'REGIAO':'id_regiao', 'UF':'id_uf', 'MASCARA':'mascara', 'ANO_MASCARA':'ano_mascara', 'DEP_ADM':'rede', 'LOCAL':'localizacao',
            'TURMA':'id_turma', 'SERIE':'serie', 'ALUNO':'id_aluno', 'DISC':'disciplina', 'CADERNO':'caderno', 'BLOCO1':'bloco_1',
            'BLOCO2':'bloco_2', 'BLOCO3':'bloco_3', 'RESP_BL1':'respostas_bloco_1', 'RESP_BL2':'respostas_bloco_2', 'RESP_BL3':'respostas_bloco_3',
            'GAB_BL1':'gabarito_bloco_1', 'GAB_BL2':'gabarito_bloco_2', 'GAB_BL3':'gabarito_bloco_3', 'UPA':'amostra', 'ESTRATO':'estrato', 
            'PESO_AC':'peso_aluno', 'PROFIC':'proficiencia_saeb', 'A081_002':'sexo', 'A081_003':'raca_cor', 'A081_006':'com_quem_mora',
            'A081_005':'escolaridade_mae', 'A081_004':'escolaridade_pai', 'A081_010':'responsaveis_conversam_escola', 'A081_013':'quantos_anos_primeiro_grau',
            'A081_014':'quantos_anos_segundo_grau', 'A081_015':'supletivo_ef', 'A081_011':'evasao_escolar_temporaria', 'A081_012':'faltas_aula', 
            'A081_009':'tempo_lazer', 'A081_007':'possui_trabalho', 'A081_008':'recebe_jornais', 'A082_002':'tempo_licao_casa', 'A082_001':'opiniao_teste', 
            'A082_006':'opiniao_frase_1', 'A082_005':'opiniao_frase_2', 'A082_007':'opiniao_frase_3', 'IDADE':'idade'}

df.rename(columns=rename95, inplace=True) #rename
df['serie'] = df['serie'].apply(lambda x: str(x).replace('8','9')) #substitui antiga 8ª série por 9º ano
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('L','LP')) #padroniza pra Língua Portuguesa
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('M','MT')) #padroniza pra Matemática
df['id_uf'] = df['id_uf'].astype(str)
df = pd.merge(df, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf']) #substitui id_uf por sigla_uf

#classificacao do desempenho no SAEB (fonte: https://intra.fundacaolemann.org.br/media/images/Screen_Shot_2021-12-08_at_10.18.42.original.png)
condicao = [
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] < 200)), 
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 200) & (df['proficiencia_saeb'] < 275)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 275) & (df['proficiencia_saeb'] < 325)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 325)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] < 225)), 
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 225) & (df['proficiencia_saeb'] < 300)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 300) & (df['proficiencia_saeb'] < 350)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado', 'Insuficiente', 'Básico', 'Proficiente', 'Avançado']

df['desempenho_aluno'] = np.nan
df['desempenho_aluno'] = np.select(condicao, valores, default=df['desempenho_aluno'])  #cria variavel classificatoria de proficiencia

df = pd.concat([df, add_df]) #cria as variaveis
df = df[ordem] #ordena

#salva particionado
for uf in ufs:
  print("Particionando {} de 1995".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=1995/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

Particionando RO de 1995
Particionando AC de 1995
Particionando AM de 1995


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando RR de 1995
Particionando PA de 1995
Particionando AP de 1995
Particionando TO de 1995
Particionando MA de 1995
Particionando PI de 1995
Particionando CE de 1995
Particionando RN de 1995
Particionando PB de 1995
Particionando PE de 1995
Particionando AL de 1995
Particionando SE de 1995
Particionando BA de 1995
Particionando MG de 1995
Particionando ES de 1995
Particionando RJ de 1995
Particionando SP de 1995
Particionando PR de 1995
Particionando SC de 1995
Particionando RS de 1995
Particionando MS de 1995
Particionando MT de 1995
Particionando GO de 1995
Particionando DF de 1995


##### **1997**

In [5]:
cie_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb97_aluno_ciencias_8serie.csv', sep=';', encoding='latin-1')
mt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb97_aluno_matematica_8serie.csv', sep=';', encoding='latin-1')
pt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb97_aluno_portugues_8serie.csv', sep=';', encoding='latin-1')
df = mt_8s.append([pt_8s, cie_8s], ignore_index=True)
del cie_8s, mt_8s, pt_8s
df['PROFIC'] = df['PROFIC'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_AC'] = df['PESO_AC'].apply(lambda x: x.replace(',','.')).astype('float64')

#desmembra data_nascimento em ano e mês, além de criar variável idade
df['mes_nascimento'] = df['A081_001'].str[:-2].replace('', np.nan).fillna(0)
df['mes_nascimento'] = df['mes_nascimento'].replace(0, '')
df['ano_nascimento'] = '19' + df['A081_001'].str[-2:].replace('', np.nan)
df['ano_nascimento'] = df['ano_nascimento'].replace('19','')

rename97 = {'ANO':'ano', 'REGIAO':'id_regiao', 'UF':'id_uf', 'ï»¿MASCARA':'mascara', 'ANO_MASCARA':'ano_mascara', 'DEP_ADM':'rede',
            'LOCAL':'localizacao', 'TURMA':'id_turma', 'TURNO':'turno', 'SERIE':'serie', 'ALUNO':'id_aluno', 'DISC':'disciplina', 'CADERNO':'caderno',
            'BLOCO1':'bloco_1', 'BLOCO2':'bloco_2', 'BLOCO3':'bloco_3', 'RESP_BL1':'respostas_bloco_1', 'RESP_BL2':'respostas_bloco_2',
            'RESP_BL3':'respostas_bloco_3', 'GAB_BL1':'gabarito_bloco_1', 'GAB_BL2':'gabarito_bloco_2', 'GAB_BL3':'gabarito_bloco_3',
            'UPA':'amostra', 'ESTRATO':'estrato', 'PESO_AC':'peso_aluno', 'PROFIC':'proficiencia_saeb', 'A081_003':'sexo',
            'A081_004':'raca_cor', 'IDADE':'idade', 'A081_005':'com_quem_mora', 'A081_010':'situacao_economica', 'A081_007':'escolaridade_mae', 
            'A081_009':'ocupacao_mae', 'A081_006':'escolaridade_pai', 'A081_008':'ocupacao_pai', 'A081_019':'responsaveis_conversam_escola',
            'A081_020':'responsaveis_comparecem_reuniao_pais', 'A081_018':'tempo_chegada_escola', 'A081_002':'idade_entrada_escola',
            'A082_007':'presenca_professor', 'A081_012':'rede_ef', 'A081_015':'reprovacao', 'A081_013':'evasao_escolar_temporaria',
            'A081_014':'motivo_evasao_escolar', 'A081_016':'faltas_aula', 'A081_017':'motivo_faltas', 'A082_008':'desempenho_prejudicado_falta',
            'A081_023':'tempo_lazer', 'A081_024':'tipo_programacao_tv', 'A081_025':'tempo_estudos', 'A081_011':'possui_trabalho',
            'A081_022':'leitura_noticias', 'A081_021':'recebe_jornais', 'A082_005':'quando_nao_entende_materia', 'A082_006':'gosta_estudar_disciplina',
            'A082_013':'entendimento_ensino', 'A082_011':'disciplina_auxilia_compreensao', 'A082_002':'tempo_licao_casa',
            'A082_004':'professor_leitura_livros_licao_casa', 'A082_003':'consulta_jornais_revistas_licao_casa', 'A081_026':'utiliza_computador',
            'A081_027':'possui_computador', 'A081_028':'utiliza_computador_escola', 'A082_014':'desempenho_disciplina', 
            'A082_010':'professor_esforca_aprendizado', 'A082_012':'professor_utiliza_espacos_externos', 'A081_030':'estuda_habitualmente', 
            'A081_029':'motivo_estar_escola', 'A082_009':'utiliza_aprendizado_diariamente', 'A082_001':'opiniao_teste'}


df.rename(columns=rename97, inplace=True) #rename
df['serie'] = df['serie'].apply(lambda x: str(x).replace('8','9')) #substitui antiga 8ª série por 9º ano
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('L','LP')) #padroniza pra Língua Portuguesa
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('M','MT')) #padroniza pra Matemática
df['id_uf'] = df['id_uf'].astype(str)
df = pd.merge(df, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf']) #substitui id_uf por sigla_uf

#classificacao do desempenho no SAEB (fonte: https://intra.fundacaolemann.org.br/media/images/Screen_Shot_2021-12-08_at_10.18.42.original.png)
condicao = [
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] < 200)), 
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 200) & (df['proficiencia_saeb'] < 275)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 275) & (df['proficiencia_saeb'] < 325)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 325)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] < 225)), 
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 225) & (df['proficiencia_saeb'] < 300)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 300) & (df['proficiencia_saeb'] < 350)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado', 'Insuficiente', 'Básico', 'Proficiente', 'Avançado']

df['desempenho_aluno'] = np.nan
df['desempenho_aluno'] = np.select(condicao, valores, default=df['desempenho_aluno'])  #cria variavel classificatoria de proficiencia

df = pd.concat([df, add_df]) #cria as variaveis
df = df[ordem] #ordena

#salva particionado
for uf in ufs:
  print("Particionando {} de 1997".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=1997/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

Particionando RO de 1997


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AC de 1997
Particionando AM de 1997
Particionando RR de 1997
Particionando PA de 1997
Particionando AP de 1997
Particionando TO de 1997
Particionando MA de 1997
Particionando PI de 1997
Particionando CE de 1997
Particionando RN de 1997
Particionando PB de 1997
Particionando PE de 1997
Particionando AL de 1997
Particionando SE de 1997
Particionando BA de 1997
Particionando MG de 1997
Particionando ES de 1997
Particionando RJ de 1997
Particionando SP de 1997
Particionando PR de 1997
Particionando SC de 1997
Particionando RS de 1997
Particionando MS de 1997
Particionando MT de 1997
Particionando GO de 1997
Particionando DF de 1997


##### **1999**

In [6]:
cie_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb99_aluno_ciencias_8serie.csv', dtype='string', sep=';', encoding='latin-1')
his_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb99_aluno_historia_8serie.csv', dtype='string', sep=';', encoding='latin-1')
geo_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb99_aluno_geografia_8serie.csv', dtype='string', sep=';', encoding='latin-1')
mt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb99_aluno_matematica_8serie.csv', dtype='string', sep=';', encoding='latin-1')
pt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb99_aluno_portugues_8serie.csv', dtype='string', sep=';', encoding='latin-1')
df = mt_8s.append([pt_8s, cie_8s, his_8s, geo_8s], ignore_index=True)
del cie_8s, mt_8s, pt_8s, his_8s, geo_8s
df['PROFIC'] = df['PROFIC'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_AC'] = df['PESO_AC'].apply(lambda x: x.replace(',','.')).astype('float64')

anos = {'2': 1981, '3': 1982, '4': 1983, '5': 1984, '6': 1985, '7': 1986}
df['ano_nascimento'] = df['A081_004'].map(anos).fillna(0).astype('int64') #substiui dicionário de anos de nascimento (2 a 7) pelo ano (INT64)
df['ano_nascimento'] = df['ano_nascimento'].replace(0,'')
fe = {'1':'1', '8':'2'}
df['faixa_etaria'] = df['A081_004'].map(fe).fillna('') #substitui dicionário de anos de nascimento (1 e 8) por faixa etária
                                                       #onde 1='19 ou mais' e 2='12 ou menos'

turno = {'Manha': 1, 'Tarde': 2, 'Noite': 3, 'Integral': 4, 'Intermedi': 5} #substitui por dicionário
df['turno'] = df['TURNO'].map(turno).fillna('')

rename99 = {'ANO':'ano', 'REGIAO':'id_regiao', 'UF':'id_uf', 'ï»¿MASCARA':'mascara', 'ANO_MASCARA':'ano_mascara', 'DEP_ADM':'rede',
            'LOCAL':'localizacao', 'TURMA':'id_turma', 'SERIE':'serie', 'ALUNO':'id_aluno', 'DISC':'disciplina', 'Caderno':'caderno',
            'Bloco1':'bloco_1', 'Bloco2':'bloco_2', 'Bloco3':'bloco_3', 'Resp_Bl1':'respostas_bloco_1', 'Resp_Bl2':'respostas_bloco_2',
            'Resp_Bl3':'respostas_bloco_3', 'Gab_Bl1':'gabarito_bloco_1', 'Gab_Bl2':'gabarito_bloco_2', 'Gab_Bl3':'gabarito_bloco_3',
            'UPA':'amostra', 'ESTRATO':'estrato', 'PESO_AC':'peso_aluno', 'PROFIC':'proficiencia_saeb', 'A081_001':'sexo', 'A081_002':'raca_cor',
            'A081_003':'mes_nascimento', 'IDADE':'idade', 'A081_005':'estado_civil', 'A081_006':'possui_filhos',
            'A081_030':'mora_mae', 'A081_029':'mora_pai', 'A081_031':'mora_irmaos', 'A081_032':'mora_conjugue', 'A081_033':'mora_filhos',
            'A081_034':'mora_outros_parentes', 'A081_035':'mora_amigos', 'A081_028':'quantidade_pessoas_domicilio',
            'A081_036':'quantidade_pessoas_despesas_domicilio', 'A081_011':'possui_moradia_rua_urbanizada', 'A081_009':'possui_agua_encanada',
            'A081_010':'possui_eletrecidade', 'A081_019':'possui_geladeira', 'A081_020':'possui_freezer', 'A081_021':'possui_maquina_lavar_roupa',
            'A081_022':'possui_aspirador_po', 'A081_016':'possui_radio', 'A081_017':'possui_tv', 'A081_018':'possui_videocassete_dvd',
            'A081_025':'possui_computador', 'A081_023':'possui_telefone', 'A081_024':'possui_celular', 'A081_027':'possui_empregada_domestica_cinco_dias',
            'A081_015':'possui_casa_dormitorio', 'A081_012':'possui_casa_cozinha', 'A081_013':'possui_casa_banheiro', 'A081_014':'possui_casa_sala',
            'A081_026':'possui_automovel', 'A081_040':'escolaridade_mae', 'A081_039':'mae_sabe_ler_escrever', 'A081_038':'escolaridade_pai',
            'A081_037':'pai_sabe_ler_escrever', 'A081_041':'responsaveis_conhecem_diretor', 'A081_042':'responsaveis_conhecem_professor', 
            'A081_043':'responsaveis_conhecem_amigo', 'A081_044':'responsaveis_conhecem_responsavel_amigo', 'A082_007':'rede_ef',
            'A082_008':'reprovacao', 'A081_007':'situacao_economica', 'A081_008':'possui_trabalho', 'A082_001':'quantidade_livros',
            'A082_003':'leitura_literatura', 'A082_002':'leitura_historia_quadrinhos', 'A082_005':'leitura_revistas_tematicas', 'A082_004':'leitura_revistas_geral',
            'A082_006':'leitura_outros_materiais', 'A082_016':'participa_gremio_escolar', 'A082_017':'participa_associacao_desportiva',
            'A082_018':'participa_associacao_moradores', 'A082_019':'participa_sindicato', 'A082_020':'participa_movimento_religioso',
            'A082_021':'participa_partido_politico', 'A082_022':'participa_ong', 'A082_023':'nao_participa', 'A082_010':'participa_atividades_esporte',
            'A082_013':'curso_atividades_artisticas', 'A082_014':'curso_reforco_escolar', 'A082_011':'curso_idioma_externo', 'A082_012':'curso_informatica',
            'A082_015':'curso_outros', 'A082_024':'gosta_estudar_disciplina', 'A082_009':'faz_licao_casa', 'A082_026':'utiliza_computador',
            'A082_027':'utiliza_computador_escola', 'A082_025':'boas_notas_disciplina'}


df.rename(columns=rename99, inplace=True) #rename
df['serie'] = df['serie'].apply(lambda x: str(x).replace('8','9')) #substitui antiga 8ª série por 9º ano
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('L','LP')) #padroniza pra Língua Portuguesa
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('M','MT')) #padroniza pra Matemática
df = pd.merge(df, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf']) #substitui id_uf por sigla_uf

#classificacao do desempenho no SAEB (fonte: https://intra.fundacaolemann.org.br/media/images/Screen_Shot_2021-12-08_at_10.18.42.original.png)
condicao = [
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] < 200)), 
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 200) & (df['proficiencia_saeb'] < 275)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 275) & (df['proficiencia_saeb'] < 325)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 325)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] < 225)), 
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 225) & (df['proficiencia_saeb'] < 300)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 300) & (df['proficiencia_saeb'] < 350)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado', 'Insuficiente', 'Básico', 'Proficiente', 'Avançado']

df['desempenho_aluno'] = np.nan
df['desempenho_aluno'] = np.select(condicao, valores, default=df['desempenho_aluno'])  #cria variavel classificatoria de proficiencia

df = pd.concat([df, add_df]) #cria as variaveis
df = df[ordem] #ordena

#salva particionado
for uf in ufs:
  print("Particionando {} de 1999".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=1999/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

Particionando RO de 1999
Particionando AC de 1999


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AM de 1999
Particionando RR de 1999
Particionando PA de 1999
Particionando AP de 1999
Particionando TO de 1999
Particionando MA de 1999
Particionando PI de 1999
Particionando CE de 1999
Particionando RN de 1999
Particionando PB de 1999
Particionando PE de 1999
Particionando AL de 1999
Particionando SE de 1999
Particionando BA de 1999
Particionando MG de 1999
Particionando ES de 1999
Particionando RJ de 1999
Particionando SP de 1999
Particionando PR de 1999
Particionando SC de 1999
Particionando RS de 1999
Particionando MS de 1999
Particionando MT de 1999
Particionando GO de 1999
Particionando DF de 1999


##### **2001**

In [7]:
mt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb01_aluno_matematica_8serie.csv', dtype='string', sep=';', encoding='latin-1')
renamemt = {'A082M006':'responsaveis_conversam_professor_diretor', 'A082M013':'responsaveis_almocam_jantam_contigo', 
            'A082M012':'responsaveis_ouvem_musica_contigo', 'A082M007':'responsaveis_conversam_livros_contigo',
            'A082M008':'responsaveis_conversam_filmes_contigo', 'A082M009':'responsaveis_conversam_programas_tv_contigo',
            'A082M014':'responsaveis_conversam_amigos', 'A082M015':'responsaveis_conversam_responsaveis_amigo',
            'A082M011':'responsaveis_conversam_outros_assuntos_contigo', 'A082M010':'responsaveis_conversam_escola',
            'A082M002':'responsaveis_conversam_boletim', 'A082M004':'responsaveis_conversam_comportamento',
            'A082M017':'responsaveis_cobram_realizacao_licao_casa', 'A082M016':'responsaveis_ajudam_realizacao_licao_casa',
            'A082M018':'responsaveis_incentivam_pontualidade', 'A082M019':'responsaveis_incentivam_boas_notas',
            'A082M001':'responsaveis_comparecem_reuniao_pais', 'A082M003':'responsaveis_participam_festas_escola',
            'A082M005':'responsaveis_participam_trabalho_voluntario', 'A082M039':'quantidade_mudancas_escola',
            'A082M033':'presenca_professor', 'A082M040':'rede_ef', 'A082M041':'reprovacao', 'A082M042':'evasao_escolar_temporaria', 'A082M043':'faltas_aula',
            'A082M020':'participa_atividades_esporte', 'A082M021':'participa_atividades_artisticas', 'A082M022':'participa_trabalho_solidario',
            'A082M023':'participa_reforco_escolar', 'A082M024':'participa_excursao_acampamento', 'A082M025':'participa_festas',
            'A082M031':'curso_atividades_artisticas', 'A082M032':'curso_reforco_escolar', 'A082M029':'curso_idioma_externo',
            'A082M030':'curso_informatica', 'A082M035':'quando_nao_entende_materia', 'A082M037':'gosta_estudar_disciplina',
            'A082M027':'professor_disciplina', 'A082M026':'faz_licao_casa', 'A082M028':'utiliza_computador_licao_casa',
            'A082M038':'comparacao_colegas_disciplina', 'A082M034':'professor_conversa_avaliacao', 'A082M036':'professor_esforca_aprendizado',
            'A082M044':'pretensao_futura'}
mt_8s.rename(columns=renamemt, inplace=True) #rename perguntas específicas do caderno de matemática

pt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb01_aluno_portugues_8serie.csv', dtype='string', sep=';', encoding='latin-1')
renamept = {'A082L001':'responsaveis_comparecem_reuniao_pais', 'A082L002':'responsaveis_conversam_boletim',
            'A082L003':'responsaveis_participam_festas_escola', 'A082L004':'responsaveis_conversam_comportamento',
            'A082L005':'responsaveis_participam_trabalho_voluntario', 'A082L006':'responsaveis_conversam_professor_diretor',
            'A082L007':'responsaveis_conversam_livros_contigo', 'A082L008':'responsaveis_conversam_filmes_contigo',
            'A082L009':'responsaveis_conversam_programas_tv_contigo', 'A082L010':'responsaveis_conversam_escola',
            'A082L011':'responsaveis_conversam_outros_assuntos_contigo', 'A082L012':'responsaveis_ouvem_musica_contigo',
            'A082L013':'responsaveis_almocam_jantam_contigo', 'A082L014':'responsaveis_conversam_amigos', 'A082L015':'responsaveis_conversam_responsaveis_amigo',
            'A082L016':'responsaveis_ajudam_realizacao_licao_casa', 'A082L017':'responsaveis_cobram_realizacao_licao_casa',
            'A082L018':'responsaveis_incentivam_pontualidade', 'A082L019':'responsaveis_incentivam_boas_notas', 'A082L020':'participa_atividades_esporte',
            'A082L021':'participa_atividades_artisticas', 'A082L022':'participa_trabalho_solidario', 'A082L023':'participa_reforco_escolar',
            'A082L024':'participa_excursao_acampamento', 'A082L025':'participa_festas', 'A082L026':'faz_licao_casa', 'A082L027':'professor_disciplina',
            'A082L028':'utiliza_computador_licao_casa', 'A082L029':'curso_idioma_externo', 'A082L030':'curso_informatica', 'A082L031':'curso_atividades_artisticas',
            'A082L032':'curso_reforco_escolar', 'A082L033':'presenca_professor', 'A082L034':'professor_conversa_avaliacao', 'A082L035':'quando_nao_entende_materia',
            'A082L036':'professor_esforca_aprendizado', 'A082L037':'gosta_estudar_disciplina', 'A082L038':'comparacao_colegas_disciplina', 'A082L039':'fez_redacao', 
            'A082L040':'quantidade_mudancas_escola', 'A082L041':'rede_ef', 'A082L042':'reprovacao', 'A082L043':'evasao_escolar_temporaria', 'A082L044':'faltas_aula',
            'A082L045':'pretensao_futura'}
pt_8s.rename(columns=renamept, inplace=True) #rename perguntas específicas do caderno de português

df = mt_8s.append(pt_8s, ignore_index=True)
del mt_8s, pt_8s
df['PROFIC'] = df['PROFIC'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_AC'] = df['PESO_AC'].apply(lambda x: x.replace(',','.')).astype('float64')

anos = {'2': 1983, '3': 1984, '4': 1985, '5': 1986, '6': 1987, '7': 1988}
df['ano_nascimento'] = df['A081_004'].map(anos).fillna(0).astype('int64') #substiui dicionário de anos de nascimento (2 a 7) pelo ano (INT64)
df['ano_nascimento'] = df['ano_nascimento'].replace(0,'')
fe = {'1':'1', '8':'2'}
df['faixa_etaria'] = df['A081_004'].map(fe).fillna('') #substitui dicionário de anos de nascimento (1 e 8) por faixa etária
                                                       #onde 1='19 ou mais' e 2='12 ou menos'

rename01 = {'ANO':'ano', 'regiao':'id_regiao', 'uf':'id_uf', 'CAP_INT':'area', 'ï»¿MASCARA':'mascara', 'ANO_MASCARA':'ano_mascara', 'DEP_ADM':'rede',
            'LOCAL':'localizacao', 'TURMA':'id_turma', 'SERIE':'serie', 'ALUNO':'id_aluno', 'DISC':'disciplina', 'CADERNO':'caderno', 'BLOCO1':'bloco_1',
            'BLOCO2':'bloco_2', 'BLOCO3':'bloco_3', 'RESP_BL1':'respostas_bloco_1', 'RESP_BL2':'respostas_bloco_2', 'RESP_BL3':'respostas_bloco_3', 
            'GAB_BL1':'gabarito_bloco_1', 'GAB_BL2':'gabarito_bloco_2', 'GAB_BL3':'gabarito_bloco_3', 'UPA':'amostra', 'ESTRATO':'estrato', 
            'PESO_AC':'peso_aluno', 'PROFIC':'proficiencia_saeb', 'A081_001':'sexo', 'A081_002':'raca_cor', 
            'A081_003':'mes_nascimento', 'A081_024':'mora_mae', 'A081_025':'mora_pai', 'A081_026':'mora_irmaos', 'A081_023':'quantidade_pessoas_domicilio',
            'A081_008':'possui_moradia_rua_urbanizada', 'A081_006':'possui_agua_encanada', 'A081_007':'possui_eletrecidade',
            'A081_008_O':'possui_eletrodomestico_sem_eletricidade', 'A081_017':'possui_geladeira', 'A081_018':'possui_freezer', 
            'A081_019':'possui_maquina_lavar_roupa', 'A081_020':'possui_aspirador_po', 'A081_014':'possui_radio', 'A081_015':'possui_tv',
            'A081_016':'possui_videocassete_dvd', 'A081_041':'possui_internet', 'A081_021':'possui_computador', 'A081_037':'possui_enciclopedia',
            'A081_038':'possui_atlas', 'A081_039':'possui_dicionario', 'A081_040':'possui_calculadora', 'A081_009':'possui_empregada_domestica', 
            'A081_012':'possui_casa_dormitorio', 'A081_010':'possui_casa_cozinha', 'A081_013':'possui_casa_banheiro', 'A081_011':'possui_casa_sala',
            'A081_022':'possui_automovel', 'A081_027':'escolaridade_mae', 'A081_028':'escolaridade_pai', 'A081_005':'possui_trabalho', 
            'A081_029':'quantidade_livros', 'A081_035':'recebe_jornais', 'A081_036':'recebe_revistas', 'A081_032':'leitura_jornais',
            'A081_031':'leitura_literatura', 'A081_030':'leitura_historia_quadrinhos', 'A081_033':'leitura_revistas_geral'}


df.rename(columns=rename01, inplace=True) #rename
df['serie'] = df['serie'].apply(lambda x: str(x).replace('8','9')) #substitui antiga 8ª série por 9º ano
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('L','LP')) #padroniza pra Língua Portuguesa
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('M','MT')) #padroniza pra Matemática
df = pd.merge(df, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf']) #substitui id_uf por sigla_uf
df['respostas_bloco_1'] = df['respostas_bloco_1'].str.upper()
df['respostas_bloco_2'] = df['respostas_bloco_2'].str.upper()
df['respostas_bloco_3'] = df['respostas_bloco_3'].str.upper()

#classificacao do desempenho no SAEB (fonte: https://intra.fundacaolemann.org.br/media/images/Screen_Shot_2021-12-08_at_10.18.42.original.png)
condicao = [
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] < 200)), 
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 200) & (df['proficiencia_saeb'] < 275)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 275) & (df['proficiencia_saeb'] < 325)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 325)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] < 225)), 
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 225) & (df['proficiencia_saeb'] < 300)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 300) & (df['proficiencia_saeb'] < 350)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado', 'Insuficiente', 'Básico', 'Proficiente', 'Avançado']

df['desempenho_aluno'] = np.nan
df['desempenho_aluno'] = np.select(condicao, valores, default=df['desempenho_aluno'])  #cria variavel classificatoria de proficiencia

#df.drop(['REDE', 'PROVA', 'A081_004', ])
df = pd.concat([df, add_df]) #cria as variaveis
df = df[ordem] #ordena

#salva particionado
for uf in ufs:
  print("Particionando {}".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2001/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, u

Particionando RO


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AC
Particionando AM
Particionando RR
Particionando PA
Particionando AP
Particionando TO
Particionando MA
Particionando PI
Particionando CE
Particionando RN
Particionando PB
Particionando PE
Particionando AL
Particionando SE
Particionando BA
Particionando MG
Particionando ES
Particionando RJ
Particionando SP
Particionando PR
Particionando SC
Particionando RS
Particionando MS
Particionando MT
Particionando GO
Particionando DF


##### **2003**

In [8]:
mt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb03_aluno_matematica_8serie.csv', dtype='string', sep=';', encoding='latin-1')
renamemt = {'A082M001':'inicio_estudos', 'A082M002':'quantidade_mudancas_escola', 'A082M003':'rede_ef', 'A082M004':'reprovacao', 
            'A082M005':'evasao_escolar_volta_antes_final_ano', 'A082M006':'evasao_escolar_temporaria', 'A082M009':'correcao_licao_casa_alunos',
            'A082M010':'gosta_estudar_disciplina', 'A082M007':'faz_licao_casa', 'A082M008':'correcao_licao_casa', 
            'A082M018':'professor_importa_voce_diz', 'A082M019':'professor_elogia_merecimento', 'A082M020':'professor_ajuda_se_necessario',
            'A082M021':'professor_atende_prontamente', 'A082M014':'professor_espera_silencio', 'A082M022':'professor_cobra', 'A082M015':'brigou_professor',
            'A082M017':'turma_exclui', 'A082M023':'familia_avisada_falta', 'A082M011':'alunos_desordeiros', 'A082M012':'alunos_atentos', 
            'A082M013':'alunos_obedientes', 'A082M016':'possui_amigos_sala', 'RESP_Bl1':'respostas_bloco_1', 'RESP_Bl2':'respostas_bloco_2',
            'RESP_Bl3':'respostas_bloco_3'}
mt_8s.rename(columns=renamemt, inplace=True) #rename perguntas específicas do caderno de matemática

pt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb03_aluno_portugues_8serie.csv', dtype='string', sep=';', encoding='latin-1')
renamept = {'A082L001':'inicio_estudos', 'A082L002':'quantidade_mudancas_escola', 'A082L003':'rede_ef', 'A082L004':'reprovacao',
            'A082L005':'evasao_escolar_volta_antes_final_ano', 'A082L006':'evasao_escolar_temporaria', 'A082L009':'correcao_licao_casa_alunos',
            'A082L010':'gosta_estudar_disciplina', 'A082L007':'faz_licao_casa', 'A082L008':'correcao_licao_casa', 
            'A082L018':'professor_importa_voce_diz', 'A082L019':'professor_elogia_merecimento', 'A082L020':'professor_ajuda_se_necessario',
            'A082L021':'professor_atende_prontamente','A082L014':'professor_espera_silencio', 'A082L022':'professor_cobra',
            'A082L015':'brigou_professor', 'A082L017':'turma_exclui', 'A082L023':'familia_avisada_falta', 'A082L011':'alunos_desordeiros',
            'A082L012':'alunos_atentos', 'A082L013':'alunos_obedientes', 'A082L016':'possui_amigos_sala', 'RESP_BL1':'respostas_bloco_1', 
            'RESP_BL2':'respostas_bloco_2', 'RESP_BL3':'respostas_bloco_3'}
pt_8s.rename(columns=renamept, inplace=True) #rename perguntas específicas do caderno de português

df = mt_8s.append(pt_8s, ignore_index=True)
del mt_8s, pt_8s
df['PROFIC'] = df['PROFIC'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_AC'] = df['PESO_AC'].apply(lambda x: x.replace(',','.')).astype('float64')

anos = {'1':1984, '2': 1985, '3': 1986, '4': 1987, '5': 1988, '6': 1989, '7': 1990}
df['ano_nascimento'] = df['A081_004'].map(anos).fillna(0).astype('int64') #substiui dicionário de anos de nascimento (2 a 7) pelo ano (INT64)
df['idade'] = (2003 - df['ano_nascimento']).replace(2003, '') #cria variavel INT64 possível para cálculo da idade
df['ano_nascimento'] = df['ano_nascimento'].replace(0,'')
fe = {'8':'2'}
df['faixa_etaria'] = df['A081_004'].map(fe).fillna('') #substitui dicionário de anos de nascimento (1 e 8) por faixa etária
                                                       #2003 não apareece a opção para nascidos antes de 1984, somente para 1991 ou depois (8 no dic)

rename03 = {'ANO':'ano', 'REGIAO':'id_regiao', 'UF':'id_uf', 'CAPITAL':'area', 'ï»¿MASCARA':'mascara', 'ANO_MASCARA':'ano_mascara', 'DEP_ADM':'rede',
            'LOCAL':'localizacao', 'TURMA':'id_turma', 'SERIE':'serie', 'ALUNO':'id_aluno', 'DISC':'disciplina', 'CADERNO':'caderno',
            'BLOCO1':'bloco_1', 'BLOCO2':'bloco_2', 'BLOCO3':'bloco_3', 'GAB_BL1':'gabarito_bloco_1', 'GAB_BL2':'gabarito_bloco_2',
            'GAB_BL3':'gabarito_bloco_3', 'UPA':'amostra', 'ESTRATO':'estrato', 'PESO_AC':'peso_aluno', 'PROFIC':'proficiencia_saeb',
            'A081_001':'sexo', 'A081_002':'raca_cor', 'A081_003':'mes_nascimento', 'A081_023':'mora_mae', 'A081_027':'mora_pai',
            'A081_022':'quantidade_pessoas_domicilio', 'A081_048':'possui_trabalho', 'A081_020':'possui_agua_encanada', 'A081_019':'possui_eletrecidade',
            'A081_013':'possui_casa_dormitorio', 'A081_012':'possui_casa_banheiro', 'A081_007':'possui_automovel', 'A081_009':'possui_geladeira',
            'A081_014':'possui_geladeira_freezer', 'A081_015':'possui_freezer', 'A081_010':'possui_maquina_lavar_roupa', 'A081_011':'possui_aspirador_po',
            'A081_006':'possui_radio', 'A081_005':'possui_tv', 'A081_008':'possui_videocassete_dvd', 'A081_016':'possui_computador', 
            'A081_017':'possui_computador_sem_internet', 'A081_021':'possui_empregada_domestica', 'A081_025':'escolaridade_mae', 
            'A081_024':'mae_sabe_ler_escrever', 'A081_026':'mae_le', 'A081_029':'escolaridade_pai', 'A081_028':'pai_sabe_ler_escrever', 'A081_030':'pai_le',
            'A081_031':'pessoa_acompanha_vida_escolar', 'A081_032':'escolaridade_pessoa', 'A081_057':'frequencia_responsaveis_conversam_diretor',
            'A081_058':'frequencia_responsaveis_conversam_professor', 'A081_050':'responsaveis_almocam_jantam_contigo', 'ALU_BE':'aluno_bolsa_escola',
            'A081_051':'responsaveis_ouvem_musica_contigo', 'A081_052':'responsaveis_conversam_livros_contigo', 'TUR_BE':'turma_bolsa_escola', 
            'A081_053':'responsaveis_conversam_filmes_contigo', 'A081_054':'responsaveis_conversam_programas_tv_contigo', 
            'A081_056':'responsaveis_conversam_amigos', 'A081_055':'responsaveis_conversam_amigos_escola', 'A081_059':'responsaveis_conversam_escola',
            'A081_061':'responsaveis_cobram_realizacao_tarefa_casa', 'A081_060':'responsaveis_ajudam_realizacao_tarefa_casa',
            'A081_062':'responsaveis_incentivam_comparecer_aulas', 'A081_063':'responsaveis_incentivam_boas_notas', 
            'A081_064':'responsaveis_comparecem_reuniao_pais', 'A081_045':'tempo_lazer', 'A081_047':'tempo_trabalho_domestico', 'A081_046':'tempo_estudos',
            'A081_018':'quantidade_livros', 'A081_033':'recebe_jornais', 'A081_034':'recebe_revistas', 'A081_037':'leitura_noticias',
            'A081_036':'leitura_literatura_infantojuvenil', 'A081_035':'leitura_historia_quadrinhos', 'A081_038':'leitura_revistas_geral',
            'A081_042':'frequenta_cinema', 'A081_044':'frequenta_espetaculo_exposicao', 'A081_041':'frequenta_teatro', 'A081_043':'frequenta_concerto',
            'A081_039':'utiliza_biblioteca_escola', 'A081_040':'utiliza_biblioteca_externa', 'A081_049':'pretensao_futura'}

df.rename(columns=rename03, inplace=True) #rename
df['serie'] = df['serie'].apply(lambda x: str(x).replace('08','9')) #substitui antiga 8ª série por 9º ano
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('L','LP')) #padroniza pra Língua Portuguesa
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('M','MT')) #padroniza pra Matemática
df = pd.merge(df, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf']) #substitui id_uf por sigla_uf

#classificacao do desempenho no SAEB (fonte: https://intra.fundacaolemann.org.br/media/images/Screen_Shot_2021-12-08_at_10.18.42.original.png)
condicao = [
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] < 200)), 
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 200) & (df['proficiencia_saeb'] < 275)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 275) & (df['proficiencia_saeb'] < 325)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 325)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] < 225)), 
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 225) & (df['proficiencia_saeb'] < 300)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 300) & (df['proficiencia_saeb'] < 350)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado', 'Insuficiente', 'Básico', 'Proficiente', 'Avançado']

df['desempenho_aluno'] = np.nan
df['desempenho_aluno'] = np.select(condicao, valores, default=df['desempenho_aluno'])  #cria variavel classificatoria de proficiencia

df = pd.concat([df, add_df]) #cria as variaveis
df = df[ordem] #ordena

#salva particionado
for uf in ufs:
  print("Particionando {} de 2003".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2003/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, u

Particionando RO de 2003


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AC de 2003
Particionando AM de 2003
Particionando RR de 2003
Particionando PA de 2003
Particionando AP de 2003
Particionando TO de 2003
Particionando MA de 2003
Particionando PI de 2003
Particionando CE de 2003
Particionando RN de 2003
Particionando PB de 2003
Particionando PE de 2003
Particionando AL de 2003
Particionando SE de 2003
Particionando BA de 2003
Particionando MG de 2003
Particionando ES de 2003
Particionando RJ de 2003
Particionando SP de 2003
Particionando PR de 2003
Particionando SC de 2003
Particionando RS de 2003
Particionando MS de 2003
Particionando MT de 2003
Particionando GO de 2003
Particionando DF de 2003


##### **2005**

In [9]:
mt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb05_aluno_matematica_8serie.csv', dtype='string', sep=';', encoding='latin-1')
renamemt = {'A082M001':'mora_mae', 'A082M002':'escolaridade_mae', 'A082M003':'mae_le', 'A082M004':'mora_pai', 'A082M005':'escolaridade_pai',
            'A082M006':'pai_le', 'A082M007':'pessoa_acompanha_vida_escolar', 'A082M008':'escolaridade_pessoa', 'A082M009':'leitura_historia_quadrinhos',
            'A082M010':'leitura_literatura_infantojuvenil', 'A082M011':'leitura_noticias', 'A082M012':'leitura_revistas_geral', 
            'A082M013':'possui_enciclopedia', 'A082M014':'possui_atlas', 'A082M015':'possui_dicionario', 'A082M016':'possui_calculadora',
            'A082M017':'responsaveis_almocam_jantam_contigo', 'A082M018':'responsaveis_ouvem_musica_contigo', 'A082M019':'responsaveis_conversam_livros_contigo',
            'A082M020':'responsaveis_conversam_filmes_contigo', 'A082M021':'responsaveis_conversam_programas_tv_contigo',
            'A082M022':'responsaveis_conversam_amigos_escola', 'A082M023':'responsaveis_conversam_amigos', 'A082M024':'frequencia_responsaveis_conversam_diretor',
            'A082M025':'frequencia_responsaveis_conversam_professor', 'A082M026':'responsaveis_conversam_escola', 
            'A082M027':'responsaveis_ajudam_realizacao_tarefa_casa', 'A082M028':'responsaveis_cobram_realizacao_tarefa_casa',
            'A082M029':'responsaveis_incentivam_comparecer_aulas', 'A082M030':'responsaveis_incentivam_boas_notas', 
            'A082M031':'responsaveis_comparecem_reuniao_pais', 'A082M032':'inicio_estudos', 'A082M033':'reprovacao', 'A082M034':'gosta_estudar_disciplina',
            'A082M035':'faz_licao_casa', 'A082M036':'correcao_licao_casa', 'A082M037':'quantidade_mudancas_escola', 'A082M038':'rede_ef',
            'A082M039':'alunos_desordeiros', 'A082M040':'alunos_atentos', 'A082M041':'alunos_obedientes', 'A082M042':'professor_espera_silencio',
            'A082M043':'possui_amigos_sala', 'A082M044':'professor_importa_voce_diz', 'A082M045':'professor_elogia_merecimento', 
            'A082M046':'professor_ajuda_se_necessario', 'A082M047':'professor_atende_prontamente', 'A082M048':'familia_avisada_falta', 
            'A082M049':'pretensao_futura', 'A082M050':'possui_trabalho', 'PROFIC_250':'proficiencia_saeb', 'PROFIC_SAE':'proficiencia'}
mt_8s.rename(columns=renamemt, inplace=True) #rename perguntas específicas do caderno de matemática

pt_8s = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb05_aluno_portugues_8serie.csv', dtype='string', sep=';', encoding='latin-1')
renamept = {'A082L001':'mora_mae', 'A082L002':'escolaridade_mae', 'A082L003':'mae_le', 'A082L004':'mora_pai', 'A082L005':'escolaridade_pai',
            'A082L006':'pai_le', 'A082L007':'pessoa_acompanha_vida_escolar', 'A082L008':'escolaridade_pessoa', 'A082L009':'leitura_historia_quadrinhos',
            'A082L010':'leitura_literatura_infantojuvenil', 'A082L011':'leitura_noticias','A082L012':'leitura_revistas_geral', 
            'A082L013':'possui_enciclopedia', 'A082L014':'possui_atlas', 'A082L015':'possui_dicionario', 'A082L016':'possui_calculadora',
            'A082L017':'responsaveis_almocam_jantam_contigo', 'A082L018':'responsaveis_ouvem_musica_contigo', 
            'A082L019':'responsaveis_conversam_livros_contigo', 'A082L020':'responsaveis_conversam_filmes_contigo', 
            'A082L021':'responsaveis_conversam_programas_tv_contigo', 'A082L022':'responsaveis_conversam_amigos_escola',
            'A082L023':'responsaveis_conversam_amigos', 'A082L024':'frequencia_responsaveis_conversam_diretor', 
            'A082L025':'frequencia_responsaveis_conversam_professor', 'A082L026':'responsaveis_conversam_escola', 
            'A082L027':'responsaveis_ajudam_realizacao_tarefa_casa', 'A082L028':'responsaveis_cobram_realizacao_tarefa_casa', 
            'A082L029':'responsaveis_incentivam_comparecer_aulas', 'A082L030':'responsaveis_incentivam_boas_notas', 
            'A082L031':'responsaveis_comparecem_reuniao_pais', 'A082L032':'inicio_estudos', 'A082L033':'reprovacao', 'A082L034':'gosta_estudar_disciplina',
            'A082L035':'faz_licao_casa', 'A082L036':'correcao_licao_casa', 'A082L037':'quantidade_mudancas_escola', 'A082L038':'rede_ef',
            'A082L039':'alunos_desordeiros', 'A082L040':'alunos_atentos', 'A082L041':'alunos_obedientes', 'A082L042':'professor_espera_silencio',
            'A082L043':'possui_amigos_sala', 'A082L044':'professor_importa_voce_diz', 'A082L045':'professor_elogia_merecimento', 
            'A082L046':'professor_ajuda_se_necessario', 'A082L047':'professor_atende_prontamente', 'A082L048':'familia_avisada_falta', 
            'A082L049':'pretensao_futura', 'A082L050':'possui_trabalho', 'PROFIC_25050':'proficiencia_saeb', 'PROFIC_SAEB':'proficiencia'}
pt_8s.rename(columns=renamept, inplace=True) #rename perguntas específicas do caderno de português

df = mt_8s.append(pt_8s, ignore_index=True)
del mt_8s, pt_8s
df['proficiencia'] = df['proficiencia'].apply(lambda x: x.replace(',','.')).astype('float64')
df['proficiencia_saeb'] = df['proficiencia_saeb'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_AT'] = df['PESO_AT'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_AC'] = df['PESO_AC'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_EC'] = df['PESO_EC'].apply(lambda x: x.replace(',','.')).astype('float64')
df['PESO_TC'] = df['PESO_TC'].apply(lambda x: x.replace(',','.')).astype('float64')

#padroniza o formato de resposta por bloco
df['respostas_bloco_1'] = df['RESP1'].str.cat(df[['RESP2','RESP3','RESP4','RESP5','RESP6','RESP7',
                                                  'RESP8','RESP9','RESP10','RESP11','RESP12','RESP13']].astype(str))
df['respostas_bloco_2'] = df['RESP14'].str.cat(df[['RESP15','RESP16','RESP17','RESP18','RESP19','RESP20',
                                                  'RESP21','RESP22','RESP23','RESP24','RESP25','RESP26']].astype(str))
df['respostas_bloco_3'] = df['RESP27'].str.cat(df[['RESP28','RESP29','RESP30','RESP31','RESP32','RESP33',
                                                  'RESP34','RESP35','RESP36','RESP37','RESP38','RESP39']].astype(str))
df.drop(['RESP1', 'RESP2','RESP3','RESP4','RESP5','RESP6','RESP7', 'RESP8','RESP9','RESP10','RESP11','RESP12','RESP13',
         'RESP14', 'RESP15','RESP16','RESP17','RESP18','RESP19','RESP20', 'RESP21','RESP22','RESP23','RESP24','RESP25','RESP26',
         'RESP27', 'RESP28','RESP29','RESP30','RESP31','RESP32','RESP33', 'RESP34','RESP35','RESP36','RESP37','RESP38','RESP39'], axis=1, inplace=True)

meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'L':11, 'M':12}
df['mes_nascimento'] = df['A081_003'].map(meses).fillna(0).astype('int64')
df['mes_nascimento'] = df['mes_nascimento'].replace(0,'')
anos = {'A':1984, 'B': 1985, 'C': 1986, 'D': 1987, 'E': 1988, 'F': 1989, 'G': 1990}
df['ano_nascimento'] = df['A081_004'].map(anos).fillna(0).astype('int64') #substiui dicionário de anos de nascimento (2 a 7) pelo ano (INT64)
df['idade'] = (2005 - df['ano_nascimento']).replace(2005, '') #cria variavel INT64 possível para cálculo da idade
df['ano_nascimento'] = df['ano_nascimento'].replace(0,'')
fe = {'H':'2'}
df['faixa_etaria'] = df['A081_004'].map(fe).fillna('') #substitui dicionário de anos de nascimento (1 e 8) por faixa etária
                                                       #2003 não apareece a opção para nascidos antes de 1984, somente para 1991 ou depois (8 no dic)

rename05 = {'ANO':'ano', 'REGIAO':'id_regiao', 'CODUF':'id_uf', 'CAPITAL':'area', 'ï»¿MASCARA':'mascara', 'ANO_MASCARA':'ano_mascara',
            'DEP_ADM':'rede', 'LOCAL':'localizacao', 'TURMA':'id_turma', 'NOMETURMA':'turma', 'TURMA_APLI':'aplicacao_turma', 'EXTRA':'turma_extra',
            'SERIE':'serie', 'ALUNO':'id_aluno', 'DISC':'disciplina', 'ALUNO_VALI':'preenchimento_caderno', 'CADERNO':'caderno', 'BLOCO1':'bloco_1',
            'BLOCO2':'bloco_2', 'BLOCO3':'bloco_3', 'GAB_BL1':'gabarito_bloco_1', 'GAB_BL2':'gabarito_bloco_2', 'GAB_BL3':'gabarito_bloco_3',
            'ESTRATO':'estrato', 'PESO_TC':'peso_escola', 'PESO_EC':'peso_turma', 'PESO_AT':'peso_aluno_turma', 'PESO_AC':'peso_aluno',
            'A081_001':'sexo', 'A081_002':'raca_cor', 'A081_022':'quantidade_pessoas_domicilio', 'A081_020':'possui_agua_encanada',
            'A081_019':'possui_eletrecidade', 'A081_013':'possui_casa_dormitorio', 'A081_012':'possui_casa_banheiro',
            'A081_007':'possui_automovel', 'A081_009':'possui_geladeira', 'A081_014':'possui_geladeira_freezer', 'A081_015':'possui_freezer',
            'A081_010':'possui_maquina_lavar_roupa', 'A081_011':'possui_aspirador_po', 'A081_006':'possui_radio', 'A081_005':'possui_tv',
            'A081_008':'possui_videocassete_dvd', 'A081_016':'possui_computador', 'A081_017':'possui_computador_sem_internet',
            'A081_021':'possui_empregada_domestica', 'A081_018':'quantidade_livros'}

df.rename(columns=rename05, inplace=True) #rename
df['serie'] = df['serie'].apply(lambda x: str(x).replace('08','9')) #substitui antiga 8ª série por 9º ano
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('L','LP')) #padroniza pra Língua Portuguesa
df['disciplina'] = df['disciplina'].apply(lambda x: str(x).replace('M','MT')) #padroniza pra Matemática
df = pd.merge(df, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf']) #substitui id_uf por sigla_uf

#classificacao do desempenho no SAEB (fonte: https://intra.fundacaolemann.org.br/media/images/Screen_Shot_2021-12-08_at_10.18.42.original.png)
condicao = [
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] < 200)), 
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 200) & (df['proficiencia_saeb'] < 275)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 275) & (df['proficiencia_saeb'] < 325)),
            ((df['disciplina'] == "LP") & (df['proficiencia_saeb'] >= 325)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] < 225)), 
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 225) & (df['proficiencia_saeb'] < 300)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 300) & (df['proficiencia_saeb'] < 350)),
            ((df['disciplina'] == "MT") & (df['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado', 'Insuficiente', 'Básico', 'Proficiente', 'Avançado']

df['desempenho_aluno'] = np.nan
df['desempenho_aluno'] = np.select(condicao, valores, default=df['desempenho_aluno'])  #cria variavel classificatoria de proficiencia

df = pd.concat([df, add_df]) #cria as variaveis
df = df[ordem] #ordena

#salva particionado
for uf in ufs:
  print("Particionando {} de 2005".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2005/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, u

Particionando RO de 2005
Particionando AC de 2005


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AM de 2005
Particionando RR de 2005
Particionando PA de 2005
Particionando AP de 2005
Particionando TO de 2005
Particionando MA de 2005
Particionando PI de 2005
Particionando CE de 2005
Particionando RN de 2005
Particionando PB de 2005
Particionando PE de 2005
Particionando AL de 2005
Particionando SE de 2005
Particionando BA de 2005
Particionando MG de 2005
Particionando ES de 2005
Particionando RJ de 2005
Particionando SP de 2005
Particionando PR de 2005
Particionando SC de 2005
Particionando RS de 2005
Particionando MS de 2005
Particionando MT de 2005
Particionando GO de 2005
Particionando DF de 2005


##### **2007**

In [10]:
#questionario dos alunos
df1 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/provabrasil07_questionario_aluno.csv', dtype='string', sep=';')
df1.rename(columns={'ID_ALUNO':'id_aluno', 'ID_SERIE':'serie', 'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao',
                      'SIGLA_UF':'sigla_uf', 'COD_MUNICIPIO':'id_municipio', 'TX_RESPOSTAS':'resp'}, inplace=True)
df1 = df1[df1.serie.isin(['8'])] #coloca exclusivamente os alunos de 9 ano
df1 = pd.merge(df1, id_reg, how='left', left_on=['sigla_uf'], right_on=['sigla_uf'])

#separa TX_RESPOSTAS nas respectivas perguntas do questionário
df1['sexo'] = df1['resp'].str[:1]
df1['raca_cor'] = df1['resp'].str[1:2]
df1['mes_nascimento'] = df1['resp'].str[2:3]
meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
df1['mes_nascimento'] = df1['mes_nascimento'].map(meses).fillna(0).astype('int64')
df1['mes_nascimento'] = df1['mes_nascimento'].replace(0,'')
df1['ano_nascimento'] = df1['resp'].str[3:4]
anos = {'B':1993, 'C': 1992, 'D': 1991, 'E': 1990, 'F': 1989, 'G': 1988}
df1['ano_nascimento'] = df1['ano_nascimento'].map(anos).fillna(0).astype('int64')
df1['idade'] = (2007 - df1['ano_nascimento']).replace(2007, '')
fe = {'A':'2', 'H':'1'}
df1['faixa_etaria'] = df1['ano_nascimento'].map(fe).fillna('')
df1['ano_nascimento'] = df1['ano_nascimento'].replace(0,'')
df1['possui_tv'] = df1['resp'].str[4:5]
df1['possui_radio'] = df1['resp'].str[5:6]
df1['possui_videocassete_dvd'] = df1['resp'].str[6:7]
df1['possui_geladeira'] = df1['resp'].str[7:8]
df1['possui_geladeira_freezer'] = df1['resp'].str[8:9]
df1['possui_maquina_lavar_roupa'] = df1['resp'].str[9:10]
df1['possui_aspirador_po'] = df1['resp'].str[10:11]
df1['possui_automovel'] = df1['resp'].str[11:12]
df1['possui_computador'] = df1['resp'].str[12:13]
df1['possui_casa_banheiro'] = df1['resp'].str[13:14]
df1['possui_empregada_domestica'] = df1['resp'].str[14:15]
df1['possui_casa_dormitorio'] = df1['resp'].str[15:16]
df1['quantidade_pessoas_domicilio'] = df1['resp'].str[16:17]
df1['mora_mae'] = df1['resp'].str[17:18]
df1['escolaridade_mae'] = df1['resp'].str[18:19]
df1['mae_sabe_ler_escrever'] = df1['resp'].str[19:20]
df1['mae_le'] = df1['resp'].str[20:21]
df1['mora_pai'] = df1['resp'].str[21:22]
df1['escolaridade_pai'] = df1['resp'].str[22:23]
df1['pai_sabe_ler_escrever'] = df1['resp'].str[23:24]
df1['pai_le'] = df1['resp'].str[24:25]
df1['responsaveis_comparecem_reuniao_pais'] = df1['resp'].str[25:26]
df1['responsaveis_incentivam_estudos'] = df1['resp'].str[26:27]
df1['responsaveis_incentivam_realizacao_licao_casa'] = df1['resp'].str[27:28]
df1['responsaveis_incentivam_leitura'] = df1['resp'].str[28:29]
df1['responsaveis_incentivam_comparecer_aulas'] = df1['resp'].str[29:30]
df1['responsaveis_conversam_escola'] = df1['resp'].str[30:31]
df1['quantidade_livros'] = df1['resp'].str[31:32]
df1['tempo_lazer'] = df1['resp'].str[32:33]
df1['tempo_trabalho_domestico'] = df1['resp'].str[33:34]
df1['possui_trabalho'] = df1['resp'].str[34:35]
df1['inicio_estudos'] = df1['resp'].str[35:36]
df1['rede_ef'] = df1['resp'].str[36:37]
df1['reprovacao'] = df1['resp'].str[37:38]
df1['evasao_escolar_ate_final_ano'] = df1['resp'].str[38:39]
df1['gosta_estudar_lp'] = df1['resp'].str[39:40]
df1['faz_licao_casa_lp'] = df1['resp'].str[40:41]
df1['correcao_licao_casa_lp'] = df1['resp'].str[41:42]
df1['gosta_estudar_mt'] = df1['resp'].str[42:43]
df1['faz_licao_casa_mt'] = df1['resp'].str[43:44]
df1['correcao_licao_casa_mt'] = df1['resp'].str[44:45]
df1['professor_elogia_merecimento'] = df1['resp'].str[45:46]
df1['pretensao_futura'] = df1['resp'].str[46:47]
df1.drop('resp', axis=1, inplace=True)

#separa o df por disciplina
df1_lp = df1.drop(['gosta_estudar_mt', 'faz_licao_casa_mt', 'correcao_licao_casa_mt'], axis=1)
df1_lp.rename(columns={'gosta_estudar_lp':'gosta_estudar_disciplina', 'faz_licao_casa_lp':'faz_licao_casa', 'correcao_licao_casa_lp':'correcao_licao_casa'}, inplace=True)
df1_mt = df1.drop(['gosta_estudar_lp', 'faz_licao_casa_lp', 'correcao_licao_casa_lp'], axis=1)
df1_mt.rename(columns={'gosta_estudar_mt':'gosta_estudar_disciplina', 'faz_licao_casa_mt':'faz_licao_casa', 'correcao_licao_casa_mt':'correcao_licao_casa'}, inplace=True)
del df1

#resultado saeb
df2 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/provabrasil07_aluno.csv', dtype='string', sep=';')
df2.rename(columns={'ID_ALUNO':'id_aluno', 'ID_TURMA':'id_turma', 'ID_SERIE':'serie', 'PK_COD_ENTIDADE':'id_escola', 'ID_DEPENDENCIA_ADM':'rede', 
                    'ID_LOCALIZACAO':'localizacao', 'SIGLA_UF':'sigla_uf', 'COD_MUNICIPIO':'id_municipio'}, inplace=True)
df2 = df2[df2.serie.isin(['8'])] #coloca exclusivamente os alunos de 9 ano

#criação da variável turno baseado no horário de início das aulas
df2['horario_inicio'] = df2['TX_HORARIO_INICIO'].apply(pd.to_numeric, errors='coerce').fillna(0).astype('int64')
condicaohr = [
            ((df2['horario_inicio'] >= 500) & (df2['horario_inicio'] <= 1059)), 
            ((df2['horario_inicio'] >= 1100) & (df2['horario_inicio'] <= 1659)),
            ((df2['horario_inicio'] >= 1700))]
valores = ['A', 'B', 'C']
df2['turno'] = np.nan
df2['turno'] = np.select(condicaohr, valores, default=df2['turno'])

#ids
ids = ['id_aluno', 'serie', 'rede', 'localizacao', 'sigla_uf', 'id_municipio']
cols = ['proficiencia', 'erro_padrao', 'proficiencia_saeb' ,'erro_padrao_saeb']

#separa o df por disciplina
df2_lp = df2.drop(['ST_MATEMATICA', 'NU_SETHETA_M', 'NU_THETAT_M', 'NU_SETHETAT_M', 'NU_THETA_M'], axis=1)
df2_lp.rename(columns={'ST_LINGUA_PORTUGUESA':'preenchimento_caderno', 'NU_THETA_L':'proficiencia', 'NU_SETHETA_L':'erro_padrao',
                       'NU_THETAT_L':'proficiencia_saeb', 'NU_SETHETAT_L':'erro_padrao_saeb'}, inplace=True)
df2_lp['disciplina'] = 'LP'
df2_lp['proficiencia_saeb'] = df2_lp['proficiencia_saeb'].apply(lambda x: x.replace(',','.')).astype('float64')
condicaolp = [
            ((df2_lp['proficiencia_saeb'] < 200)), 
            ((df2_lp['proficiencia_saeb'] >= 200) & (df2_lp['proficiencia_saeb'] < 275)),
            ((df2_lp['proficiencia_saeb'] >= 275) & (df2_lp['proficiencia_saeb'] < 325)),
            ((df2_lp['proficiencia_saeb'] >= 325))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
df2_lp['desempenho_aluno'] = np.nan #cria variavel classificatoria de proficiencia
df2_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=df2_lp['desempenho_aluno'])

df_lp = pd.merge(df1_lp, df2_lp, how='right', left_on=ids, right_on=ids) #merge entre questionário e resultado
del df1_lp, df2_lp

df2_mt = df2.drop(['ST_LINGUA_PORTUGUESA', 'NU_SETHETA_L', 'NU_THETAT_L', 'NU_SETHETAT_L', 'NU_THETA_L'], axis=1)
df2_mt.rename(columns={'ST_MATEMATICA':'preenchimento_caderno', 'NU_THETA_M':'proficiencia', 'NU_SETHETA_M':'erro_padrao',
                       'NU_THETAT_M':'proficiencia_saeb', 'NU_SETHETAT_M':'erro_padrao_saeb'}, inplace=True)
df2_mt['disciplina'] = 'MT'
df2_mt['proficiencia_saeb'] = df2_mt['proficiencia_saeb'].apply(lambda x: x.replace(',','.')).astype('float64')
condicaomt = [
            ((df2_mt['proficiencia_saeb'] < 225)), 
            ((df2_mt['proficiencia_saeb'] >= 225) & (df2_mt['proficiencia_saeb'] < 300)),
            ((df2_mt['proficiencia_saeb'] >= 300) & (df2_mt['proficiencia_saeb'] < 350)),
            ((df2_mt['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
df2_mt['desempenho_aluno'] = np.nan #cria variavel classificatoria de proficiencia
df2_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=df2_mt['desempenho_aluno'])
df_mt = pd.merge(df1_mt, df2_mt, how='right', left_on=ids, right_on=ids) #substitui id_uf por sigla_uf
del df1_mt, df2_mt, df2

df = df_mt.append(df_lp, ignore_index=True)
del df_mt, df_lp

df['erro_padrao'] = df['erro_padrao'].apply(lambda x: x.replace(',','.')).astype('float64')
df['erro_padrao_saeb'] = df['erro_padrao_saeb'].apply(lambda x: x.replace(',','.')).astype('float64')
df['proficiencia'] = df['proficiencia'].apply(lambda x: x.replace(',','.')).astype('float64')
df['serie'] = df['serie'].apply(lambda x: str(x).replace('8','9')) #substitui antiga 8ª série por 9º ano

#salva particionado
for uf in ufs:
  print("Particionando {} de 2007".format(uf))
  df_uf = df[df['sigla_uf'] == uf]
  df_uf = pd.concat([df_uf, add_df]) #cria as variaveis
  df_uf = df_uf[ordem] #ordena
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2007/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

Particionando RO de 2007


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AC de 2007
Particionando AM de 2007
Particionando RR de 2007
Particionando PA de 2007
Particionando AP de 2007
Particionando TO de 2007
Particionando MA de 2007
Particionando PI de 2007
Particionando CE de 2007
Particionando RN de 2007
Particionando PB de 2007
Particionando PE de 2007
Particionando AL de 2007
Particionando SE de 2007
Particionando BA de 2007
Particionando MG de 2007
Particionando ES de 2007
Particionando RJ de 2007
Particionando SP de 2007
Particionando PR de 2007
Particionando SC de 2007
Particionando RS de 2007
Particionando MS de 2007
Particionando MT de 2007
Particionando GO de 2007
Particionando DF de 2007


##### **2009**

In [4]:
#questionario dos alunos
df1 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/provabrasil09_quest_aluno.txt', dtype='string', sep=';')
df1.rename(columns={'ID_ALUNO':'id_aluno', 'ID_SERIE':'serie', 'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao',
                      'SIGLA_UF':'sigla_uf', 'COD_MUNICIPIO':'id_municipio', 'TX_RESPOSTAS':'resp'}, inplace=True)
df1 = df1[df1.serie.isin(['9'])] #coloca exclusivamente os alunos de 9 ano
df1 = pd.merge(df1, id_reg, how='left', left_on=['sigla_uf'], right_on=['sigla_uf'])

#separa TX_RESPOSTAS nas respectivas perguntas do questionário
df1['sexo'] = df1['resp'].str[:1]
df1['raca_cor'] = df1['resp'].str[1:2]
df1['mes_nascimento'] = df1['resp'].str[2:3]
meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
df1['mes_nascimento'] = df1['mes_nascimento'].map(meses).fillna(0).astype('int64')
df1['mes_nascimento'] = df1['mes_nascimento'].replace(0,'')
df1['ano_nascimento'] = df1['resp'].str[3:4]
anos = {'B':1993, 'C': 1992, 'D': 1991, 'E': 1990, 'F': 1989, 'G': 1988}
df1['ano_nascimento'] = df1['ano_nascimento'].map(anos).fillna(0).astype('int64')
df1['idade'] = (2009 - df1['ano_nascimento']).replace(2009, '')
fe = {'A':'2', 'H':'1'}
df1['faixa_etaria'] = df1['ano_nascimento'].map(fe).fillna('')
df1['ano_nascimento'] = df1['ano_nascimento'].replace(0,'')
df1['possui_tv'] = df1['resp'].str[4:5]
df1['possui_radio'] = df1['resp'].str[5:6]
df1['possui_videocassete_dvd'] = df1['resp'].str[6:7]
df1['possui_geladeira'] = df1['resp'].str[7:8]
df1['possui_geladeira_freezer'] = df1['resp'].str[8:9]
df1['possui_freezer'] = df1['resp'].str[9:10]
df1['possui_maquina_lavar_roupa'] = df1['resp'].str[10:11]
df1['possui_automovel'] = df1['resp'].str[11:12]
df1['possui_computador'] = df1['resp'].str[12:13]
df1['possui_casa_banheiro'] = df1['resp'].str[13:14]
df1['possui_empregada_domestica'] = df1['resp'].str[14:15]
df1['possui_casa_dormitorio'] = df1['resp'].str[15:16]
df1['quantidade_pessoas_domicilio'] = df1['resp'].str[16:17]
df1['mora_mae'] = df1['resp'].str[17:18]
df1['escolaridade_mae'] = df1['resp'].str[18:19]
df1['mae_sabe_ler_escrever'] = df1['resp'].str[19:20]
df1['mae_le'] = df1['resp'].str[20:21]
df1['mora_pai'] = df1['resp'].str[21:22]
df1['escolaridade_pai'] = df1['resp'].str[22:23]
df1['pai_sabe_ler_escrever'] = df1['resp'].str[23:24]
df1['pai_le'] = df1['resp'].str[24:25]
df1['responsaveis_comparecem_reuniao_pais'] = df1['resp'].str[25:26]
df1['responsaveis_incentivam_estudos'] = df1['resp'].str[26:27]
df1['responsaveis_incentivam_realizacao_licao_casa'] = df1['resp'].str[27:28]
df1['responsaveis_incentivam_leitura'] = df1['resp'].str[28:29]
df1['responsaveis_incentivam_comparecer_aulas'] = df1['resp'].str[29:30]
df1['responsaveis_conversam_escola'] = df1['resp'].str[30:31]
df1['quantidade_livros'] = df1['resp'].str[31:32]
df1['tempo_lazer'] = df1['resp'].str[32:33]
df1['tempo_trabalho_domestico'] = df1['resp'].str[33:34]
df1['possui_trabalho'] = df1['resp'].str[34:35]
df1['inicio_estudos'] = df1['resp'].str[35:36]
df1['rede_ef'] = df1['resp'].str[36:37]
df1['reprovacao'] = df1['resp'].str[37:38]
df1['evasao_escolar_ate_final_ano'] = df1['resp'].str[38:39]
df1['gosta_estudar_lp'] = df1['resp'].str[39:40]
df1['faz_licao_casa_lp'] = df1['resp'].str[40:41]
df1['correcao_licao_casa_lp'] = df1['resp'].str[41:42]
df1['gosta_estudar_mt'] = df1['resp'].str[42:43]
df1['faz_licao_casa_mt'] = df1['resp'].str[43:44]
df1['correcao_licao_casa_mt'] = df1['resp'].str[44:45]
df1['professor_elogia_merecimento'] = df1['resp'].str[45:46]
df1['pretensao_futura'] = df1['resp'].str[46:47]
df1.drop('resp', axis=1, inplace=True)

#separa o df por disciplina
df1_lp = df1.drop(['gosta_estudar_mt', 'faz_licao_casa_mt', 'correcao_licao_casa_mt'], axis=1)
df1_lp.rename(columns={'gosta_estudar_lp':'gosta_estudar_disciplina', 'faz_licao_casa_lp':'faz_licao_casa', 'correcao_licao_casa_lp':'correcao_licao_casa'}, inplace=True)
df1_mt = df1.drop(['gosta_estudar_lp', 'faz_licao_casa_lp', 'correcao_licao_casa_lp'], axis=1)
df1_mt.rename(columns={'gosta_estudar_mt':'gosta_estudar_disciplina', 'faz_licao_casa_mt':'faz_licao_casa', 'correcao_licao_casa_mt':'correcao_licao_casa'}, inplace=True)
del df1

#resultado saeb
df2 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/provabrasil09_aluno.txt', dtype='string', sep=';')
df2.rename(columns={'ID_ALUNO':'id_aluno', 'ID_TURMA':'id_turma', 'ID_SERIE':'serie', 'PK_COD_ENTIDADE':'id_escola', 'ID_DEPENDENCIA_ADM':'rede', 
                    'ID_LOCALIZACAO':'localizacao', 'SIGLA_UF':'sigla_uf', 'COD_MUNICIPIO':'id_municipio', 'CO_TURNO':'turno'}, inplace=True)
df2 = df2[df2.serie.isin(['9'])] #coloca exclusivamente os alunos de 9 ano

#ids
ids = ['id_aluno', 'serie', 'rede', 'localizacao', 'sigla_uf', 'id_municipio']

#separa o df por disciplina
df2_lp = df2.drop(['ST_MATEMATICA', 'NU_SETHETA_M', 'NU_THETAT_M', 'NU_SETHETAT_M', 'NU_THETA_M'], axis=1)
df2_lp.rename(columns={'ST_LINGUA_PORTUGUESA':'preenchimento_caderno', 'NU_THETA_L':'proficiencia', 'NU_SETHETA_L':'erro_padrao',
                       'NU_THETAT_L':'proficiencia_saeb', 'NU_SETHETAT_L':'erro_padrao_saeb'}, inplace=True)
df2_lp['disciplina'] = 'LP'
df2_lp['proficiencia_saeb'] = df2_lp['proficiencia_saeb'].astype('float64')
condicaolp = [
            ((df2_lp['proficiencia_saeb'] < 200)), 
            ((df2_lp['proficiencia_saeb'] >= 200) & (df2_lp['proficiencia_saeb'] < 275)),
            ((df2_lp['proficiencia_saeb'] >= 275) & (df2_lp['proficiencia_saeb'] < 325)),
            ((df2_lp['proficiencia_saeb'] >= 325))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
df2_lp['desempenho_aluno'] = np.nan #cria variavel classificatoria de proficiencia
df2_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=df2_lp['desempenho_aluno'])

df_lp = pd.merge(df1_lp, df2_lp, how='right', left_on=ids, right_on=ids) #merge entre questionário e resultado
del df1_lp, df2_lp

df2_mt = df2.drop(['ST_LINGUA_PORTUGUESA', 'NU_SETHETA_L', 'NU_THETAT_L', 'NU_SETHETAT_L', 'NU_THETA_L'], axis=1)
df2_mt.rename(columns={'ST_MATEMATICA':'preenchimento_caderno', 'NU_THETA_M':'proficiencia', 'NU_SETHETA_M':'erro_padrao',
                       'NU_THETAT_M':'proficiencia_saeb', 'NU_SETHETAT_M':'erro_padrao_saeb'}, inplace=True)
df2_mt['disciplina'] = 'MT'
df2_mt['proficiencia_saeb'] = df2_mt['proficiencia_saeb'].astype('float64')
condicaomt = [
            ((df2_mt['proficiencia_saeb'] < 225)), 
            ((df2_mt['proficiencia_saeb'] >= 225) & (df2_mt['proficiencia_saeb'] < 300)),
            ((df2_mt['proficiencia_saeb'] >= 300) & (df2_mt['proficiencia_saeb'] < 350)),
            ((df2_mt['proficiencia_saeb'] >= 350))]
valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
df2_mt['desempenho_aluno'] = np.nan #cria variavel classificatoria de proficiencia
df2_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=df2_mt['desempenho_aluno'])
df_mt = pd.merge(df1_mt, df2_mt, how='right', left_on=ids, right_on=ids) #substitui id_uf por sigla_uf
del df1_mt, df2_mt, df2

#salva particionado
for uf in ufs:
  print("Particionando {} de 2009".format(uf))
  df_uf = df_mt[df_mt['sigla_uf'] == uf]
  df_uf = pd.concat([df_uf, add_df])
  df_uf = df_uf[ordem]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2009/sigla_uf={}/microdados_mt.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
  df_uf = df_lp[df_lp['sigla_uf'] == uf]
  df_uf = pd.concat([df_uf, add_df])
  df_uf = df_uf[ordem]
  df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2009/sigla_uf={}/microdados_lp.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

Particionando RO de 2009
Particionando AC de 2009
Particionando AM de 2009


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando RR de 2009
Particionando PA de 2009
Particionando AP de 2009
Particionando TO de 2009
Particionando MA de 2009
Particionando PI de 2009
Particionando CE de 2009
Particionando RN de 2009
Particionando PB de 2009
Particionando PE de 2009
Particionando AL de 2009
Particionando SE de 2009
Particionando BA de 2009
Particionando MG de 2009
Particionando ES de 2009
Particionando RJ de 2009
Particionando SP de 2009
Particionando PR de 2009
Particionando SC de 2009
Particionando RS de 2009
Particionando MS de 2009
Particionando MT de 2009
Particionando GO de 2009
Particionando DF de 2009


##### **2011**

In [4]:
#df1: questionario dos alunos
renamedf1 = {'ID_SAEB':'ano', 'ID_REGIAO':'id_regiao', 'ID_UF':'id_uf', 'ID_MUNICIPIO':'id_municipio', 'ID_ESCOLA':'id_escola',  
             'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao', 'ID_TURMA':'id_turma', 'ID_TURNO':'turno', 'ID_SERIE':'serie',
             'ID_ALUNO':'id_aluno', 'IN_SITUACAO_CENSO':'situacao_censo', 'IN_PROVA_BRASIL':'indicador_prova_brasil', 
             'IN_PREENCHIMENTO':'preenchimento_questionario','TX_RESP_Q001':'sexo', 'TX_RESP_Q002':'raca_cor', 'TX_RESP_Q018':'mora_mae',
             'TX_RESP_Q022':'mora_pai', 'TX_RESP_Q017':'quantidade_pessoas_domicilio', 'TX_RESP_Q046':'possui_trabalho', 'TX_RESP_Q016':'possui_casa_dormitorio',
             'TX_RESP_Q014':'possui_casa_banheiro', 'TX_RESP_Q012':'possui_automovel', 'TX_RESP_Q008':'possui_geladeira', 'TX_RESP_Q009':'possui_geladeira_freezer',
             'TX_RESP_Q010':'possui_freezer', 'TX_RESP_Q011':'possui_maquina_lavar_roupa', 'TX_RESP_Q006':'possui_radio', 'TX_RESP_Q005':'possui_tv',
             'TX_RESP_Q007':'possui_videocassete_dvd', 'TX_RESP_Q013':'possui_computador', 'TX_RESP_Q015':'possui_empregada_domestica',
             'TX_RESP_Q019':'escolaridade_mae', 'TX_RESP_Q020':'mae_sabe_ler_escrever', 'TX_RESP_Q021':'mae_le', 'TX_RESP_Q023':'escolaridade_pai',
             'TX_RESP_Q024':'pai_sabe_ler_escrever', 'TX_RESP_Q025':'pai_le', 'TX_RESP_Q031':'responsaveis_conversam_escola', 
             'TX_RESP_Q027':'responsaveis_incentivam_estudos', 'TX_RESP_Q029':'responsaveis_incentivam_leitura', 
             'TX_RESP_Q028':'responsaveis_incentivam_realizacao_licao_casa', 'TX_RESP_Q030':'responsaveis_incentivam_comparecer_aulas',
             'TX_RESP_Q026':'responsaveis_comparecem_reuniao_pais', 'TX_RESP_Q047':'inicio_estudos', 'TX_RESP_Q048':'rede_ef', 'TX_RESP_Q049':'reprovacao',
             'TX_RESP_Q050':'evasao_escolar_ate_final_ano', 'TX_RESP_Q044':'tempo_lazer', 'TX_RESP_Q045':'tempo_trabalho_domestico', 'TX_RESP_Q032':'leitura_noticias',
             'TX_RESP_Q033':'leitura_livros_geral', 'TX_RESP_Q034':'leitura_literatura_infantojuvenil', 'TX_RESP_Q036':'leitura_historia_quadrinhos',
             'TX_RESP_Q037':'leitura_revista_comportamento', 'TX_RESP_Q035':'leitura_revistas_geral', 'TX_RESP_Q038':'leitura_internet',
             'TX_RESP_Q039':'frequenta_biblioteca', 'TX_RESP_Q040':'frequenta_cinema', 'TX_RESP_Q041':'frequenta_teatro', 'TX_RESP_Q042':'frequenta_concerto',
             'TX_RESP_Q043':'frequenta_festas_comunidade', 'TX_RESP_Q057':'utiliza_biblioteca_escola', 'TX_RESP_Q058':'pretensao_futura',
             'TX_RESP_Q051':'gosta_estudar_lp', 'TX_RESP_Q054':'gosta_estudar_mt', 'TX_RESP_Q052':'faz_licao_casa_lp', 'TX_RESP_Q055':'faz_licao_casa_mt',
             'TX_RESP_Q053':'correcao_licao_casa_lp', 'TX_RESP_Q056':'correcao_licao_casa_mt'}

df1 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb11_quest_aluno.csv', dtype='string', sep=';')
df1 = df1[df1.ID_SERIE.isin(['9'])] #coloca exclusivamente os alunos de 9 ano
df1.rename(columns=renamedf1, inplace=True)
df1 = pd.merge(df1, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])
meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
df1['mes_nascimento'] = df1['TX_RESP_Q003'].map(meses).fillna(0).astype('int64')
df1['mes_nascimento'] = df1['mes_nascimento'].replace(0,'')
anos = {'B':1998, 'C': 1997, 'D': 1996, 'E': 1995, 'F': 1994, 'G': 1993}
df1['ano_nascimento'] = df1['TX_RESP_Q004'].map(anos).fillna(0).astype('int64')
df1['idade'] = (2011 - df1['ano_nascimento']).replace(2011, '')
df1['ano_nascimento'] = df1['ano_nascimento'].replace(0,'')
fe = {'A':'2', 'H':'1'}
df1['faixa_etaria'] = df1['TX_RESP_Q004'].map(fe).fillna('')
df1.drop(['ID_CAPITAL', 'TX_RESP_Q003', 'TX_RESP_Q004', 'TX_RESP_Q059', 'TX_RESP_Q060', 'TX_RESP_Q061', 'TX_RESP_Q062'], axis=1, inplace=True)
#separa o df por disciplina e UF
for uf in ufs:
  df1_uf = df1[df1['sigla_uf'] == uf]
  df1_lp = df1_uf.drop(['gosta_estudar_mt', 'faz_licao_casa_mt', 'correcao_licao_casa_mt'], axis=1)
  df1_lp.rename(columns={'gosta_estudar_lp':'gosta_estudar_disciplina', 'faz_licao_casa_lp':'faz_licao_casa', 'correcao_licao_casa_lp':'correcao_licao_casa'}, inplace=True)
  df1_lp['disciplina'] = 'LP'
  exec("df1_lp.to_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/questionario_lp.csv', index=False, float_format='%.0f', encoding='utf-8', na_rep='')".format(uf))
  del df1_lp
  df1_mt = df1_uf.drop(['gosta_estudar_lp', 'faz_licao_casa_lp', 'correcao_licao_casa_lp'], axis=1)
  df1_mt.rename(columns={'gosta_estudar_mt':'gosta_estudar_disciplina', 'faz_licao_casa_mt':'faz_licao_casa', 'correcao_licao_casa_mt':'correcao_licao_casa'}, inplace=True)
  df1_mt['disciplina'] = 'MT'
  exec("df1_mt.to_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/questionario_mt.csv', index=False, float_format='%.0f', encoding='utf-8', na_rep='')".format(uf))
  del df1_mt
del df1


#df2: resultado dos alunos
renamedf2 = {'ID_SAEB':'ano', 'ID_REGIAO':'id_regiao', 'ID_UF':'id_uf', 'ID_MUNICIPIO':'id_municipio', 'ID_ESCOLA':'id_escola',  
             'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao', 'ID_TURMA':'id_turma', 'ID_TURNO':'turno', 'ID_SERIE':'serie',
             'ID_ALUNO':'id_aluno', 'IN_SITUACAO_CENSO':'situacao_censo', 'IN_PROVA_BRASIL':'indicador_prova_brasil', 
             'IN_PREENCHIMENTO':'preenchimento_questionario', 'IN_PROFICIENCIA':'indicador_proficiencia', 'PESO':'peso_aluno'}

df2 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb11_resultado_aluno.csv', dtype='string', sep=';', nrows=100)
df2 = df2[df2.ID_SERIE.isin(['9'])] #coloca exclusivamente os alunos de 9 ano
df2.rename(columns=renamedf2, inplace=True)
df2.drop('ID_CAPITAL', axis=1, inplace=True)
df2['peso_aluno'] = df2['peso_aluno'].apply(lambda x: x.replace(',','.'))
df2['PROFICIENCIA_LP'] = df2['PROFICIENCIA_LP'].apply(lambda x: x.replace(',','.'))
df2['DESVIO_PADRAO_LP'] = df2['DESVIO_PADRAO_LP'].apply(lambda x: x.replace(',','.'))
df2['PROFICIENCIA_LP_SAEB'] = df2['PROFICIENCIA_LP_SAEB'].apply(lambda x: x.replace(',','.'))
df2['DESVIO_PADRAO_LP_SAEB'] = df2['DESVIO_PADRAO_LP_SAEB'].apply(lambda x: x.replace(',','.'))
df2['PROFICIENCIA_MT'] = df2['PROFICIENCIA_MT'].apply(lambda x: x.replace(',','.'))
df2['DESVIO_PADRAO_MT'] = df2['DESVIO_PADRAO_MT'].apply(lambda x: x.replace(',','.'))
df2['PROFICIENCIA_MT_SAEB'] = df2['PROFICIENCIA_MT_SAEB'].apply(lambda x: x.replace(',','.'))
df2['DESVIO_PADRAO_MT_SAEB'] = df2['DESVIO_PADRAO_MT_SAEB'].apply(lambda x: x.replace(',','.'))
df2 = pd.merge(df2, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])
#separa o df por disciplina e UF
for uf in ufs:
  df2_uf = df2[df2['sigla_uf'] == uf]
  df2_lp = df2_uf.drop(['PROFICIENCIA_MT', 'DESVIO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 'DESVIO_PADRAO_MT_SAEB'], axis=1)
  df2_lp.rename(columns={'PROFICIENCIA_LP':'proficiencia', 'DESVIO_PADRAO_LP':'erro_padrao', 
                         'PROFICIENCIA_LP_SAEB':'proficiencia_saeb', 'DESVIO_PADRAO_LP_SAEB':'erro_padrao_saeb'}, inplace=True)
  df2_lp['proficiencia_saeb'] = pd.to_numeric(df2_lp['proficiencia_saeb'], errors='coerce').astype('float64')
  condicaolp = [
            ((df2_lp['proficiencia_saeb'] < 200)), 
            ((df2_lp['proficiencia_saeb'] >= 200) & (df2_lp['proficiencia_saeb'] < 275)),
            ((df2_lp['proficiencia_saeb'] >= 275) & (df2_lp['proficiencia_saeb'] < 325)),
            ((df2_lp['proficiencia_saeb'] >= 325))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  df2_lp['desempenho_aluno'] = np.nan #cria variavel classificatoria de proficiencia
  df2_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=df2_lp['desempenho_aluno'])
  df2_lp['disciplina'] = 'LP'
  exec("df2_lp.to_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/resultado_lp.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
  del df2_lp
  df2_mt = df2_uf.drop(['PROFICIENCIA_LP', 'DESVIO_PADRAO_LP', 'PROFICIENCIA_LP_SAEB', 'DESVIO_PADRAO_LP_SAEB'], axis=1)
  df2_mt.rename(columns={'PROFICIENCIA_MT':'proficiencia', 'DESVIO_PADRAO_MT':'erro_padrao', 
                         'PROFICIENCIA_MT_SAEB':'proficiencia_saeb', 'DESVIO_PADRAO_MT_SAEB':'erro_padrao_saeb'}, inplace=True)
  df2_mt['proficiencia_saeb'] = pd.to_numeric(df2_mt['proficiencia_saeb'], errors='coerce').astype('float64')
  condicaomt = [
            ((df2_mt['proficiencia_saeb'] < 225)), 
            ((df2_mt['proficiencia_saeb'] >= 225) & (df2_mt['proficiencia_saeb'] < 300)),
            ((df2_mt['proficiencia_saeb'] >= 300) & (df2_mt['proficiencia_saeb'] < 350)),
            ((df2_mt['proficiencia_saeb'] >= 350))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  df2_mt['desempenho_aluno'] = np.nan #cria variavel classificatoria de proficiencia
  df2_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=df2_mt['desempenho_aluno'])
  df2_mt['disciplina'] = 'MT'
  exec("df2_mt.to_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/resultado_mt.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
  del df2_mt
del df2

#df3: respostas dos alunos
renamedf3 = {'ID_SAEB':'ano', 'ID_REGIAO':'id_regiao', 'ID_UF':'id_uf', 'ID_MUNICIPIO':'id_municipio', 'ID_ESCOLA':'id_escola',  
             'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao', 'ID_TURMA':'id_turma', 'ID_TURNO':'turno', 'ID_SERIE':'serie',
             'ID_ALUNO':'id_aluno', 'IN_SITUACAO_CENSO':'situacao_censo', 'IN_PROVA_BRASIL':'indicador_prova_brasil', 
             'IN_PREENCHIMENTO':'preenchimento_questionario', 'IN_PROFICIENCIA':'indicador_proficiencia', 'ID_CADERNO':'caderno',
             'ID_BLOCO1':'bloco_1', 'ID_BLOCO2':'bloco_2'}

df3 = pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb11_resposta_aluno.csv', dtype='string', sep=';')
df3 = df3[df3.ID_SERIE.isin(['9'])] #coloca exclusivamente os alunos de 9 ano
df3.rename(columns=renamedf3, inplace=True)
df3.drop('ID_CAPITAL', axis=1, inplace=True)
df3 = pd.merge(df3, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])
#separa o df por disciplina e UF
for uf in ufs:
  df3_uf = df3[df3['sigla_uf'] == uf]
  df3_lp = df3_uf.drop(['TX_RESP_BLOCO_1_MT', 'TX_RESP_BLOCO_2_MT'], axis=1)
  df3_lp.rename(columns={'TX_RESP_BLOCO_1_LP':'respostas_bloco_1', 'TX_RESP_BLOCO_2_LP':'respostas_bloco_2'}, inplace=True)
  df3_lp['disciplina'] = 'LP'
  exec("df3_lp.to_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/resposta_lp.csv', index=False, float_format='%.0f', encoding='utf-8', na_rep='')".format(uf))
  del df3_lp
  df3_mt = df3_uf.drop(['TX_RESP_BLOCO_1_LP', 'TX_RESP_BLOCO_2_LP'], axis=1)
  df3_mt.rename(columns={'TX_RESP_BLOCO_1_MT':'respostas_bloco_1', 'TX_RESP_BLOCO_2_MT':'respostas_bloco_2'}, inplace=True)
  df3_mt['disciplina'] = 'MT'
  exec("df3_mt.to_csv('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/resposta_mt.csv', index=False, float_format='%.0f', encoding='utf-8', na_rep='')".format(uf))
  del df3_mt
del df3

#ids
ids = ['ano', 'id_regiao', 'id_uf', 'sigla_uf', 'id_municipio', 'id_escola', 'rede','localizacao', 'id_turma', 'turno', 'serie', 'id_aluno',
       'situacao_censo', 'indicador_prova_brasil', 'preenchimento_questionario', 'disciplina']

for uf in ufs:
  exec("os.chdir('/content/gdrive/MyDrive/br_inep_saeb/temp/aluno_ef_9ano/ano=2011/sigla_uf={}/')".format(uf))
  lp = [i for i in glob.glob('*_lp.csv')]
  dfs = [pd.read_csv(f) for f in lp]
  df_lp = reduce(lambda left,right: pd.merge(left,right,on=ids), dfs)
  df_lp = pd.concat([df_lp, add_df]) #cria as variaveis
  df_lp = df_lp[ordem]
  df_lp.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_lp.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2011/sigla_uf={}/microdados_lp.csv', index=False, float_format='%.0f', encoding='utf-8', na_rep='')".format(uf))
  del df_lp
  mt = [i for i in glob.glob('*_mt.csv')]
  dfs = [pd.read_csv(f) for f in mt]
  df_mt = reduce(lambda left,right: pd.merge(left,right,on=ids), dfs)
  df_mt = pd.concat([pd.read_csv(f) for f in mt])
  df_mt = pd.concat([df_mt, add_df]) #cria as variaveis
  df_mt = df_mt[ordem]
  df_mt.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
  exec("df_mt.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2011/sigla_uf={}/microdados_mt.csv', index=False, float_format='%.0f', encoding='utf-8', na_rep='')".format(uf))
  del df_mt

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipy

##### **2013**

In [4]:
df_mt = pd.DataFrame()
df_lp = pd.DataFrame()

rename13 = {'ID_PROVA_BRASIL':'ano', 'ID_REGIAO':'id_regiao', 'ID_UF':'id_uf', 'ID_MUNICIPIO':'id_municipio', 'ID_AREA':'area',
            'ID_ESCOLA':'id_escola', 'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao', 'ID_TURMA':'id_turma', 'ID_TURNO':'turno',
            'ID_SERIE':'serie', 'ID_ALUNO':'id_aluno', 'IN_SITUACAO_CENSO':'situacao_censo', 'IN_PREENCHIMENTO_PROVA':'preenchimento_caderno',
            'ID_CADERNO':'caderno', 'ID_BLOCO_1':'bloco_1', 'ID_BLOCO_2':'bloco_2', 'IN_PROFICIENCIA':'indicador_proficiencia',
            'IN_PROVA_BRASIL':'indicador_prova_brasil', 'ESTRATO_ANEB':'estrato', 'IN_PREENCHIMENTO_QUESTIONARIO':'preenchimento_questionario',
            'TX_RESP_Q001':'sexo', 'TX_RESP_Q002':'raca_cor', 'TX_RESP_Q018':'mora_mae', 'TX_RESP_Q022':'mora_pai', 
            'TX_RESP_Q016':'quantidade_pessoas_domicilio', 'TX_RESP_Q045':'possui_trabalho', 'TX_RESP_Q015':'possui_casa_dormitorio', 
            'TX_RESP_Q014':'possui_casa_banheiro', 'TX_RESP_Q012':'possui_automovel', 'TX_RESP_Q008':'possui_geladeira',
            'TX_RESP_Q009':'possui_geladeira_freezer', 'TX_RESP_Q010':'possui_freezer', 'TX_RESP_Q011':'possui_maquina_lavar_roupa',
            'TX_RESP_Q006':'possui_radio', 'TX_RESP_Q005':'possui_tv', 'TX_RESP_Q007':'possui_videocassete_dvd', 'TX_RESP_Q013':'possui_computador',
            'TX_RESP_Q017':'possui_empregada_domestica_cinco_dias', 'TX_RESP_Q019':'escolaridade_mae', 'TX_RESP_Q020':'mae_sabe_ler_escrever',
            'TX_RESP_Q021':'mae_le', 'TX_RESP_Q023':'escolaridade_pai', 'TX_RESP_Q024':'pai_sabe_ler_escrever', 'TX_RESP_Q025':'pai_le',
            'TX_RESP_Q031':'responsaveis_conversam_escola', 'TX_RESP_Q027':'responsaveis_incentivam_estudos',
            'TX_RESP_Q029':'responsaveis_incentivam_leitura', 'TX_RESP_Q028':'responsaveis_incentivam_realizacao_licao_casa',
            'TX_RESP_Q030':'responsaveis_incentivam_comparecer_aulas', 'TX_RESP_Q026':'responsaveis_comparecem_reuniao_pais',
            'TX_RESP_Q046':'inicio_estudos', 'TX_RESP_Q047':'rede_ef', 'TX_RESP_Q048':'reprovacao', 'TX_RESP_Q049':'evasao_escolar_ate_final_ano',
            'TX_RESP_Q043':'tempo_lazer', 'TX_RESP_Q044':'tempo_trabalho_domestico', 'TX_RESP_Q032':'leitura_noticias',
            'TX_RESP_Q033':'leitura_livros_geral', 'TX_RESP_Q034':'leitura_literatura_infantojuvenil', 'TX_RESP_Q036':'leitura_historia_quadrinhos',
            'TX_RESP_Q037':'leitura_revista_comportamento', 'TX_RESP_Q035':'leitura_revistas_geral', 'TX_RESP_Q038':'leitura_internet',
            'TX_RESP_Q039':'frequenta_biblioteca', 'TX_RESP_Q040':'frequenta_cinema', 'TX_RESP_Q041':'frequenta_espetaculo_exposicao',
            'TX_RESP_Q042':'frequenta_festas_comunidade', 'TX_RESP_Q056':'utiliza_biblioteca_escola', 'TX_RESP_Q057':'pretensao_futura'}

for chunk in pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb13_aluno_9ano.csv', dtype='string', sep=',', chunksize = 1000000):
  chunk.rename(columns=rename13, inplace=True)
  chunk = pd.merge(chunk, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])
  meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
  chunk['mes_nascimento'] = chunk['TX_RESP_Q003'].map(meses).fillna(0).astype('int64')
  chunk['mes_nascimento'] = chunk['mes_nascimento'].replace(0,'')
  anos = {'B':2000, 'C': 1999, 'D': 1998, 'E': 1997, 'F': 1996, 'G': 1995}
  chunk['ano_nascimento'] = chunk['TX_RESP_Q004'].map(anos).fillna(0).astype('int64')
  chunk['idade'] = (2013 - chunk['ano_nascimento']).replace(2013, '')
  fe = {'A':'2', 'H':'1'}
  chunk['faixa_etaria'] = chunk['TX_RESP_Q004'].map(fe).fillna('')
  chunk['ano_nascimento'] = chunk['ano_nascimento'].replace(0, '')
  chunk_mt = chunk.drop(['TX_RESP_BLOCO_1_LP', 'TX_RESP_BLOCO_2_LP', 'PESO_ALUNO_LP', 'PROFICIENCIA_LP', 'DESVIO_PADRAO_LP', 'PROFICIENCIA_LP_SAEB', 'DESVIO_PADRAO_LP_SAEB',
                         'TX_RESP_Q050', 'TX_RESP_Q051', 'TX_RESP_Q052'], axis=1)
  chunk_lp = chunk.drop(['TX_RESP_BLOCO_1_MT', 'TX_RESP_BLOCO_2_MT', 'PESO_ALUNO_MT', 'PROFICIENCIA_MT', 'DESVIO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 'DESVIO_PADRAO_MT_SAEB',
                         'TX_RESP_Q053', 'TX_RESP_Q054', 'TX_RESP_Q055'], axis=1)
  chunk_lp.rename(columns={'TX_RESP_BLOCO_1_LP':'respostas_bloco_1', 'TX_RESP_BLOCO_2_LP':'respostas_bloco_2', 'PESO_ALUNO_LP':'peso_aluno', 'PROFICIENCIA_LP':'proficiencia', 
                          'DESVIO_PADRAO_LP':'erro_padrao', 'PROFICIENCIA_LP_SAEB':'proficiencia_saeb', 'DESVIO_PADRAO_LP_SAEB':'erro_padrao_saeb',
                          'TX_RESP_Q050':'gosta_estudar_disciplina', 'TX_RESP_Q051':'faz_licao_casa', 'TX_RESP_Q052':'correcao_licao_casa'}, inplace=True)
  chunk_mt.rename(columns={'TX_RESP_BLOCO_1_MT':'respostas_bloco_1', 'TX_RESP_BLOCO_2_MT':'respostas_bloco_2', 'PESO_ALUNO_MT':'peso_aluno', 'PROFICIENCIA_MT':'proficiencia', 
                          'DESVIO_PADRAO_MT':'erro_padrao', 'PROFICIENCIA_MT_SAEB':'proficiencia_saeb', 'DESVIO_PADRAO_MT_SAEB':'erro_padrao_saeb',
                          'TX_RESP_Q053':'gosta_estudar_disciplina', 'TX_RESP_Q054':'faz_licao_casa', 'TX_RESP_Q055':'correcao_licao_casa'}, inplace=True)
  chunk_mt['disciplina'] = 'MT'
  chunk_lp['disciplina'] = 'LP'
  chunk_mt['proficiencia_saeb'] = chunk_mt['proficiencia_saeb'].astype('float64')
  condicaomt = [
            ((chunk_mt['proficiencia_saeb'] < 225)), 
            ((chunk_mt['proficiencia_saeb'] >= 225) & (chunk_mt['proficiencia_saeb'] < 300)),
            ((chunk_mt['proficiencia_saeb'] >= 300) & (chunk_mt['proficiencia_saeb'] < 350)),
            ((chunk_mt['proficiencia_saeb'] >= 350))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  chunk_mt['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
  chunk_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=chunk_mt['desempenho_aluno'])
  chunk_lp['proficiencia_saeb'] = chunk_lp['proficiencia_saeb'].astype('float64')
  condicaolp = [
            ((chunk_lp['proficiencia_saeb'] < 200)), 
            ((chunk_lp['proficiencia_saeb'] >= 200) & (chunk_lp['proficiencia_saeb'] < 275)),
            ((chunk_lp['proficiencia_saeb'] >= 275) & (chunk_lp['proficiencia_saeb'] < 325)),
            ((chunk_lp['proficiencia_saeb'] >= 325))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  chunk_lp['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
  chunk_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=chunk_lp['desempenho_aluno'])
  df_mt = df_mt.append(chunk_mt)
  df_lp = df_lp.append(chunk_lp)
  for uf in ufs:
    print("Particionando {} de 2013".format(uf))
    df_uf = df_mt[df_mt['sigla_uf'] == uf]
    df_uf = pd.concat([df_uf, add_df])
    df_uf = df_uf[ordem]
    df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
    exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2013/sigla_uf={}/microdados_mt.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
    df_uf = df_lp[df_lp['sigla_uf'] == uf]
    df_uf = pd.concat([df_uf, add_df])
    df_uf = df_uf[ordem]
    df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
    exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2013/sigla_uf={}/microdados_lp.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
    

##### **2015**

In [5]:
df_mt = pd.DataFrame()
df_lp = pd.DataFrame()

for chunk in pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb15_aluno_9ano.csv', dtype='string', sep=',', chunksize = 1000000):
  chunk.rename(columns=rename13, inplace=True)
  chunk = pd.merge(chunk, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])
  meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
  chunk['mes_nascimento'] = chunk['TX_RESP_Q003'].map(meses).fillna(0).astype('int64')
  chunk['mes_nascimento'] = chunk['mes_nascimento'].replace(0,'')
  anos = {'B':2002, 'C': 2001, 'D': 2000, 'E': 1999, 'F': 1998, 'G': 1997}
  chunk['ano_nascimento'] = chunk['TX_RESP_Q004'].map(anos).fillna(0).astype('int64')
  chunk['idade'] = (2015 - chunk['ano_nascimento']).replace(2015, '')
  fe = {'A':'2', 'H':'1'}
  chunk['faixa_etaria'] = chunk['TX_RESP_Q004'].map(fe).fillna('')
  chunk['ano_nascimento'] = chunk['ano_nascimento'].replace(0, '')
  chunk_mt = chunk.drop(['TX_RESP_BLOCO_1_LP', 'TX_RESP_BLOCO_2_LP', 'PESO_ALUNO_LP', 'PROFICIENCIA_LP', 'DESVIO_PADRAO_LP', 'PROFICIENCIA_LP_SAEB', 'DESVIO_PADRAO_LP_SAEB',
                         'TX_RESP_Q050', 'TX_RESP_Q051', 'TX_RESP_Q052'], axis=1)
  chunk_lp = chunk.drop(['TX_RESP_BLOCO_1_MT', 'TX_RESP_BLOCO_2_MT', 'PESO_ALUNO_MT', 'PROFICIENCIA_MT', 'DESVIO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 'DESVIO_PADRAO_MT_SAEB',
                         'TX_RESP_Q053', 'TX_RESP_Q054', 'TX_RESP_Q055'], axis=1)
  chunk_lp.rename(columns={'TX_RESP_BLOCO_1_LP':'respostas_bloco_1', 'TX_RESP_BLOCO_2_LP':'respostas_bloco_2', 'PESO_ALUNO_LP':'peso_aluno', 'PROFICIENCIA_LP':'proficiencia', 
                          'DESVIO_PADRAO_LP':'erro_padrao', 'PROFICIENCIA_LP_SAEB':'proficiencia_saeb', 'DESVIO_PADRAO_LP_SAEB':'erro_padrao_saeb',
                          'TX_RESP_Q050':'gosta_estudar_disciplina', 'TX_RESP_Q051':'faz_licao_casa', 'TX_RESP_Q052':'correcao_licao_casa'}, inplace=True)
  chunk_mt.rename(columns={'TX_RESP_BLOCO_1_MT':'respostas_bloco_1', 'TX_RESP_BLOCO_2_MT':'respostas_bloco_2', 'PESO_ALUNO_MT':'peso_aluno', 'PROFICIENCIA_MT':'proficiencia', 
                          'DESVIO_PADRAO_MT':'erro_padrao', 'PROFICIENCIA_MT_SAEB':'proficiencia_saeb', 'DESVIO_PADRAO_MT_SAEB':'erro_padrao_saeb',
                          'TX_RESP_Q053':'gosta_estudar_disciplina', 'TX_RESP_Q054':'faz_licao_casa', 'TX_RESP_Q055':'correcao_licao_casa'}, inplace=True)
  chunk_mt['disciplina'] = 'MT'
  chunk_lp['disciplina'] = 'LP'
  chunk_mt['proficiencia_saeb'] = chunk_mt['proficiencia_saeb'].astype('float64')
  condicaomt = [
            ((chunk_mt['proficiencia_saeb'] < 225)), 
            ((chunk_mt['proficiencia_saeb'] >= 225) & (chunk_mt['proficiencia_saeb'] < 300)),
            ((chunk_mt['proficiencia_saeb'] >= 300) & (chunk_mt['proficiencia_saeb'] < 350)),
            ((chunk_mt['proficiencia_saeb'] >= 350))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  chunk_mt['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
  chunk_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=chunk_mt['desempenho_aluno'])
  chunk_lp['proficiencia_saeb'] = chunk_lp['proficiencia_saeb'].astype('float64')
  condicaolp = [
            ((chunk_lp['proficiencia_saeb'] < 200)), 
            ((chunk_lp['proficiencia_saeb'] >= 200) & (chunk_lp['proficiencia_saeb'] < 275)),
            ((chunk_lp['proficiencia_saeb'] >= 275) & (chunk_lp['proficiencia_saeb'] < 325)),
            ((chunk_lp['proficiencia_saeb'] >= 325))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  chunk_lp['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
  chunk_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=chunk_lp['desempenho_aluno'])
  df_mt = df_mt.append(chunk_mt)
  df_lp = df_lp.append(chunk_lp)
  for uf in ufs:
    print("Particionando {} de 2015".format(uf))
    df_uf = df_mt[df_mt['sigla_uf'] == uf]
    df_uf = pd.concat([df_uf, add_df])
    df_uf = df_uf[ordem]
    df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
    exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2015/sigla_uf={}/microdados_mt.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
    df_uf = df_lp[df_lp['sigla_uf'] == uf]
    df_uf = pd.concat([df_uf, add_df])
    df_uf = df_uf[ordem]
    df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
    exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2015/sigla_uf={}/microdados_lp.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

Particionando RO de 2015


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AC de 2015
Particionando AM de 2015
Particionando RR de 2015
Particionando PA de 2015
Particionando AP de 2015
Particionando TO de 2015
Particionando MA de 2015
Particionando PI de 2015
Particionando CE de 2015
Particionando RN de 2015
Particionando PB de 2015
Particionando PE de 2015
Particionando AL de 2015
Particionando SE de 2015
Particionando BA de 2015
Particionando MG de 2015
Particionando ES de 2015
Particionando RJ de 2015
Particionando SP de 2015
Particionando PR de 2015
Particionando SC de 2015
Particionando RS de 2015
Particionando MS de 2015
Particionando MT de 2015
Particionando GO de 2015
Particionando DF de 2015
Particionando RO de 2015
Particionando AC de 2015
Particionando AM de 2015
Particionando RR de 2015
Particionando PA de 2015
Particionando AP de 2015
Particionando TO de 2015
Particionando MA de 2015
Particionando PI de 2015
Particionando CE de 2015
Particionando RN de 2015
Particionando PB de 2015
Particionando PE de 2015
Particionando AL de 2015


##### **2017**

In [5]:
df_mt = pd.DataFrame()
df_lp = pd.DataFrame()

for chunk in pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb17_aluno_9ano.csv', dtype='string', sep=',', chunksize = 1000000):
  chunk.rename(columns=rename13, inplace=True)
  chunk = pd.merge(chunk, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])
  meses = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12}
  chunk['mes_nascimento'] = chunk['TX_RESP_Q003'].map(meses).fillna(0).astype('int64')
  chunk['mes_nascimento'] = chunk['mes_nascimento'].replace(0,'')
  anos = {'B':2004, 'C': 2003, 'D': 2002, 'E': 2001, 'F': 2000, 'G': 1999}
  chunk['ano_nascimento'] = chunk['TX_RESP_Q004'].map(anos).fillna(0).astype('int64')
  chunk['idade'] = (2017 - chunk['ano_nascimento']).replace(2017, '')
  fe = {'A':'2', 'H':'1'}
  chunk['faixa_etaria'] = chunk['TX_RESP_Q004'].map(fe).fillna('')
  chunk['ano_nascimento'] = chunk['ano_nascimento'].replace(0, '')
  chunk_mt = chunk.drop(['TX_RESP_BLOCO_1_LP', 'TX_RESP_BLOCO_2_LP', 'PESO_ALUNO_LP', 'PROFICIENCIA_LP', 'ERRO_PADRAO_LP', 'PROFICIENCIA_LP_SAEB', 
                         'ERRO_PADRAO_LP_SAEB', 'TX_RESP_Q050', 'TX_RESP_Q051', 'TX_RESP_Q052'], axis=1)
  chunk_lp = chunk.drop(['TX_RESP_BLOCO_1_MT', 'TX_RESP_BLOCO_2_MT', 'PESO_ALUNO_MT', 'PROFICIENCIA_MT', 'ERRO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 
                         'ERRO_PADRAO_MT_SAEB', 'TX_RESP_Q053', 'TX_RESP_Q054', 'TX_RESP_Q055'], axis=1)
  chunk_lp.rename(columns={'TX_RESP_BLOCO_1_LP':'respostas_bloco_1', 'TX_RESP_BLOCO_2_LP':'respostas_bloco_2', 'PESO_ALUNO_LP':'peso_aluno', 'PROFICIENCIA_LP':'proficiencia', 
                          'ERRO_PADRAO_LP':'erro_padrao', 'PROFICIENCIA_LP_SAEB':'proficiencia_saeb', 'ERRO_PADRAO_LP_SAEB':'erro_padrao_saeb',
                          'TX_RESP_Q050':'gosta_estudar_disciplina', 'TX_RESP_Q051':'faz_licao_casa', 'TX_RESP_Q052':'correcao_licao_casa'}, inplace=True)
  chunk_mt.rename(columns={'TX_RESP_BLOCO_1_MT':'respostas_bloco_1', 'TX_RESP_BLOCO_2_MT':'respostas_bloco_2', 'PESO_ALUNO_MT':'peso_aluno', 'PROFICIENCIA_MT':'proficiencia', 
                          'ERRO_PADRAO_MT':'erro_padrao', 'PROFICIENCIA_MT_SAEB':'proficiencia_saeb', 'ERRO_PADRAO_MT_SAEB':'erro_padrao_saeb',
                          'TX_RESP_Q053':'gosta_estudar_disciplina', 'TX_RESP_Q054':'faz_licao_casa', 'TX_RESP_Q055':'correcao_licao_casa'}, inplace=True)
  chunk_mt['disciplina'] = 'MT'
  chunk_lp['disciplina'] = 'LP'
  chunk_mt['proficiencia_saeb'] = chunk_mt['proficiencia_saeb'].astype('float64')
  condicaomt = [
            ((chunk_mt['proficiencia_saeb'] < 225)), 
            ((chunk_mt['proficiencia_saeb'] >= 225) & (chunk_mt['proficiencia_saeb'] < 300)),
            ((chunk_mt['proficiencia_saeb'] >= 300) & (chunk_mt['proficiencia_saeb'] < 350)),
            ((chunk_mt['proficiencia_saeb'] >= 350))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  chunk_mt['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
  chunk_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=chunk_mt['desempenho_aluno'])
  chunk_lp['proficiencia_saeb'] = chunk_lp['proficiencia_saeb'].astype('float64')
  condicaolp = [
            ((chunk_lp['proficiencia_saeb'] < 200)), 
            ((chunk_lp['proficiencia_saeb'] >= 200) & (chunk_lp['proficiencia_saeb'] < 275)),
            ((chunk_lp['proficiencia_saeb'] >= 275) & (chunk_lp['proficiencia_saeb'] < 325)),
            ((chunk_lp['proficiencia_saeb'] >= 325))]
  valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
  chunk_lp['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
  chunk_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=chunk_lp['desempenho_aluno'])
  df_mt = df_mt.append(chunk_mt)
  df_lp = df_lp.append(chunk_lp)
  for uf in ufs:
    print("Particionando {} de 2017".format(uf))
    df_uf = df_mt[df_mt['sigla_uf'] == uf]
    df_uf = pd.concat([df_uf, add_df])
    df_uf = df_uf[ordem]
    df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
    exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2017/sigla_uf={}/microdados_mt.csv', index=False, encoding='utf-8', na_rep='')".format(uf))
    df_uf = df_lp[df_lp['sigla_uf'] == uf]
    df_uf = pd.concat([df_uf, add_df])
    df_uf = df_uf[ordem]
    df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
    exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano=2017/sigla_uf={}/microdados_lp.csv', index=False, encoding='utf-8', na_rep='')".format(uf))

Particionando RO de 2017
Particionando AC de 2017


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando AM de 2017
Particionando RR de 2017
Particionando PA de 2017
Particionando AP de 2017
Particionando TO de 2017
Particionando MA de 2017
Particionando PI de 2017
Particionando CE de 2017
Particionando RN de 2017
Particionando PB de 2017
Particionando PE de 2017
Particionando AL de 2017
Particionando SE de 2017
Particionando BA de 2017
Particionando MG de 2017
Particionando ES de 2017
Particionando RJ de 2017
Particionando SP de 2017
Particionando PR de 2017
Particionando SC de 2017
Particionando RS de 2017
Particionando MS de 2017
Particionando MT de 2017
Particionando GO de 2017
Particionando DF de 2017
Particionando RO de 2017
Particionando AC de 2017
Particionando AM de 2017
Particionando RR de 2017
Particionando PA de 2017
Particionando AP de 2017
Particionando TO de 2017
Particionando MA de 2017
Particionando PI de 2017
Particionando CE de 2017
Particionando RN de 2017
Particionando PB de 2017
Particionando PE de 2017
Particionando AL de 2017
Particionando SE de 2017


##### **2019**

In [4]:
ano = [2019]
dropmt = ['IN_PREENCHIMENTO_LP', 'IN_PREENCHIMENTO_CH', 'IN_PREENCHIMENTO_CN', 'IN_PRESENCA_LP', 'IN_PRESENCA_CH', 'IN_PRESENCA_CN', 
           'ID_CADERNO_LP', 'ID_BLOCO_1_LP', 'ID_BLOCO_2_LP', 'ID_CADERNO_CH', 'ID_BLOCO_1_CH', 'ID_BLOCO_2_CH', 'ID_BLOCO_3_CH', 
           'NU_BLOCO_1_ABERTA_CH', 'NU_BLOCO_2_ABERTA_CH', 'ID_CADERNO_CN', 'ID_BLOCO_1_CN', 'ID_BLOCO_2_CN', 'ID_BLOCO_3_CN', 'NU_BLOCO_1_ABERTA_CN', 
           'NU_BLOCO_2_ABERTA_CN', 'TX_RESP_BLOCO1_LP', 'TX_RESP_BLOCO2_LP', 'TX_RESP_BLOCO1_CH', 'TX_RESP_BLOCO2_CH', 'TX_RESP_BLOCO3_CH', 'CO_CONCEITO_Q1_CH',
           'CO_CONCEITO_Q2_CH', 'TX_RESP_BLOCO1_CN', 'TX_RESP_BLOCO2_CN', 'TX_RESP_BLOCO3_CN', 'CO_CONCEITO_Q1_CN', 'CO_CONCEITO_Q2_CN', 'IN_PROFICIENCIA_LP',
            'IN_PROFICIENCIA_CH', 'IN_PROFICIENCIA_CN', 'PESO_ALUNO_LP', 'PROFICIENCIA_LP', 'ERRO_PADRAO_LP', 'PROFICIENCIA_LP_SAEB',
           'ERRO_PADRAO_LP_SAEB', 'PESO_ALUNO_CH', 'PROFICIENCIA_CH', 'ERRO_PADRAO_CH', 'PROFICIENCIA_CH_SAEB', 'ERRO_PADRAO_CH_SAEB', 
           'PESO_ALUNO_CN', 'PROFICIENCIA_CN', 'ERRO_PADRAO_CN', 'PROFICIENCIA_CN_SAEB', 'ERRO_PADRAO_CN_SAEB']
renamemt = {'IN_PREENCHIMENTO_MT':'preenchimento_caderno', 'IN_PRESENCA_MT':'presenca', 'ID_CADERNO_MT':'caderno', 'ID_BLOCO_1_MT':'bloco_1',
             'ID_BLOCO_2_MT':'bloco_2', 'TX_RESP_BLOCO1_MT':'respostas_bloco_1', 'TX_RESP_BLOCO2_MT':'respostas_bloco_2', 
             'IN_PROFICIENCIA_MT':'indicador_proficiencia', 'PESO_ALUNO_MT':'peso_aluno', 'PROFICIENCIA_MT':'proficiencia', 
             'ERRO_PADRAO_MT':'erro_padrao', 'PROFICIENCIA_MT_SAEB':'proficiencia_saeb', 'ERRO_PADRAO_MT_SAEB':'erro_padrao_saeb'}

droplp = ['IN_PREENCHIMENTO_MT', 'IN_PREENCHIMENTO_CH', 'IN_PREENCHIMENTO_CN', 'IN_PRESENCA_MT', 'IN_PRESENCA_CH', 'IN_PRESENCA_CN', 'ID_CADERNO_MT', 
           'ID_BLOCO_1_MT', 'ID_BLOCO_2_MT', 'ID_CADERNO_CH', 'ID_BLOCO_1_CH', 'ID_BLOCO_2_CH', 'ID_BLOCO_3_CH', 'NU_BLOCO_1_ABERTA_CH', 'NU_BLOCO_2_ABERTA_CH', 
           'ID_CADERNO_CN', 'ID_BLOCO_1_CN', 'ID_BLOCO_2_CN', 'ID_BLOCO_3_CN', 'NU_BLOCO_1_ABERTA_CN', 'NU_BLOCO_2_ABERTA_CN', 'TX_RESP_BLOCO1_MT', 
           'TX_RESP_BLOCO2_MT', 'TX_RESP_BLOCO1_CH', 'TX_RESP_BLOCO2_CH', 'TX_RESP_BLOCO3_CH', 'CO_CONCEITO_Q1_CH', 'CO_CONCEITO_Q2_CH', 'TX_RESP_BLOCO1_CN', 
           'TX_RESP_BLOCO2_CN', 'TX_RESP_BLOCO3_CN', 'CO_CONCEITO_Q1_CN', 'CO_CONCEITO_Q2_CN', 'IN_PROFICIENCIA_MT', 'IN_PROFICIENCIA_CH', 'IN_PROFICIENCIA_CN', 
           'PESO_ALUNO_MT', 'PROFICIENCIA_MT', 'ERRO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 'ERRO_PADRAO_MT_SAEB', 'PESO_ALUNO_CH',
           'PROFICIENCIA_CH', 'ERRO_PADRAO_CH', 'PROFICIENCIA_CH_SAEB', 'ERRO_PADRAO_CH_SAEB', 'PESO_ALUNO_CN', 'PROFICIENCIA_CN', 'ERRO_PADRAO_CN',
           'PROFICIENCIA_CN_SAEB', 'ERRO_PADRAO_CN_SAEB']
renamelp = {'IN_PREENCHIMENTO_LP':'preenchimento_caderno', 'IN_PRESENCA_LP':'presenca', 'ID_CADERNO_LP':'caderno', 'ID_BLOCO_1_LP':'bloco_1',
             'ID_BLOCO_2_LP':'bloco_2', 'TX_RESP_BLOCO1_LP':'respostas_bloco_1', 'TX_RESP_BLOCO2_LP':'respostas_bloco_2', 
             'IN_PROFICIENCIA_LP':'indicador_proficiencia', 'PESO_ALUNO_LP':'peso_aluno', 'PROFICIENCIA_LP':'proficiencia', 
             'ERRO_PADRAO_LP':'erro_padrao', 'PROFICIENCIA_LP_SAEB':'proficiencia_saeb', 'ERRO_PADRAO_LP_SAEB':'erro_padrao_saeb'}

dropch = ['IN_PREENCHIMENTO_LP', 'IN_PREENCHIMENTO_MT', 'IN_PREENCHIMENTO_CN', 'IN_PRESENCA_LP', 'IN_PRESENCA_MT',
           'IN_PRESENCA_CN', 'ID_CADERNO_LP', 'ID_BLOCO_1_LP', 'ID_BLOCO_2_LP', 'ID_CADERNO_MT', 'ID_BLOCO_1_MT', 'ID_BLOCO_2_MT',
           'ID_CADERNO_CN', 'ID_BLOCO_1_CN', 'ID_BLOCO_2_CN', 'ID_BLOCO_3_CN', 'NU_BLOCO_1_ABERTA_CN', 'NU_BLOCO_2_ABERTA_CN', 'TX_RESP_BLOCO1_LP', 
           'TX_RESP_BLOCO2_LP', 'TX_RESP_BLOCO1_MT', 'TX_RESP_BLOCO2_MT', 'TX_RESP_BLOCO1_CN', 'TX_RESP_BLOCO2_CN', 'TX_RESP_BLOCO3_CN', 'CO_CONCEITO_Q1_CN', 
           'CO_CONCEITO_Q2_CN', 'IN_PROFICIENCIA_LP', 'IN_PROFICIENCIA_MT', 'IN_PROFICIENCIA_CN', 'PESO_ALUNO_LP', 'PROFICIENCIA_LP', 'ERRO_PADRAO_LP', 
           'PROFICIENCIA_LP_SAEB', 'ERRO_PADRAO_LP_SAEB', 'PESO_ALUNO_MT', 'PROFICIENCIA_MT', 'ERRO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 'ERRO_PADRAO_MT_SAEB', 
           'PESO_ALUNO_CN', 'PROFICIENCIA_CN', 'ERRO_PADRAO_CN', 'PROFICIENCIA_CN_SAEB', 'ERRO_PADRAO_CN_SAEB']
renamech = {'IN_PREENCHIMENTO_CH':'preenchimento_caderno', 'IN_PRESENCA_CH':'presenca', 'ID_CADERNO_CH':'caderno', 'ID_BLOCO_1_CH':'bloco_1',
             'ID_BLOCO_2_CH':'bloco_2', 'ID_BLOCO_3_CH':'bloco_3', 'NU_BLOCO_1_ABERTA_CH':'bloco_1_aberto', 'NU_BLOCO_2_ABERTA_CH':'bloco_2_aberto',
             'TX_RESP_BLOCO1_CH':'respostas_bloco_1', 'TX_RESP_BLOCO2_CH':'respostas_bloco_2', 'TX_RESP_BLOCO3_CH':'respostas_bloco_3',
             'CO_CONCEITO_Q1_CH':'conceito_q1', 'CO_CONCEITO_Q2_CH':'conceito_q2', 'IN_PROFICIENCIA_CH':'indicador_proficiencia',
             'PESO_ALUNO_CH':'peso_aluno', 'PROFICIENCIA_CH':'proficiencia', 'ERRO_PADRAO_CH':'erro_padrao', 'PROFICIENCIA_CH_SAEB':'proficiencia_saeb',
             'ERRO_PADRAO_CH_SAEB':'erro_padrao_saeb'}           

dropcn = ['IN_PREENCHIMENTO_LP', 'IN_PREENCHIMENTO_MT', 'IN_PREENCHIMENTO_CH', 'IN_PRESENCA_LP', 'IN_PRESENCA_MT',
           'IN_PRESENCA_CH', 'ID_CADERNO_LP', 'ID_BLOCO_1_LP', 'ID_BLOCO_2_LP', 'ID_CADERNO_MT', 'ID_BLOCO_1_MT', 'ID_BLOCO_2_MT',
           'ID_CADERNO_CH', 'ID_BLOCO_1_CH', 'ID_BLOCO_2_CH', 'ID_BLOCO_3_CH', 'NU_BLOCO_1_ABERTA_CH', 'NU_BLOCO_2_ABERTA_CH', 'TX_RESP_BLOCO1_LP', 
           'TX_RESP_BLOCO2_LP', 'TX_RESP_BLOCO1_MT', 'TX_RESP_BLOCO2_MT', 'TX_RESP_BLOCO1_CH', 'TX_RESP_BLOCO2_CH', 'TX_RESP_BLOCO3_CH', 'CO_CONCEITO_Q1_CH',
           'CO_CONCEITO_Q2_CH', 'IN_PROFICIENCIA_LP', 'IN_PROFICIENCIA_MT', 'IN_PROFICIENCIA_CH', 'PESO_ALUNO_LP', 'PROFICIENCIA_LP', 'ERRO_PADRAO_LP', 
           'PROFICIENCIA_LP_SAEB', 'ERRO_PADRAO_LP_SAEB', 'PESO_ALUNO_MT', 'PROFICIENCIA_MT', 'ERRO_PADRAO_MT', 'PROFICIENCIA_MT_SAEB', 'ERRO_PADRAO_MT_SAEB',
           'PESO_ALUNO_CH', 'PROFICIENCIA_CH', 'ERRO_PADRAO_CH', 'PROFICIENCIA_CH_SAEB', 'ERRO_PADRAO_CH_SAEB']
renamecn = {'IN_PREENCHIMENTO_CN':'preenchimento_caderno', 'IN_PRESENCA_CN':'presenca', 'ID_CADERNO_CN':'caderno', 'ID_BLOCO_1_CN':'bloco_1',
             'ID_BLOCO_2_CN':'bloco_2', 'ID_BLOCO_3_CN':'bloco_3', 'NU_BLOCO_1_ABERTA_CN':'bloco_aberto_1', 'NU_BLOCO_2_ABERTA_CN':'bloco_aberto_2',
             'TX_RESP_BLOCO1_CN':'respostas_bloco_1', 'TX_RESP_BLOCO2_CN':'respostas_bloco_2', 'TX_RESP_BLOCO3_CN':'respostas_bloco_3', 
             'CO_CONCEITO_Q1_CN':'conceito_q1', 'CO_CONCEITO_Q2_CN':'conceito_q2', 'IN_PROFICIENCIA_CN':'indicador_proficiencia',
             'PESO_ALUNO_CN':'peso_aluno', 'PROFICIENCIA_CN':'proficiencia', 'ERRO_PADRAO_CN':'erro_padrao', 'PROFICIENCIA_CN_SAEB':'proficiencia_saeb',
             'ERRO_PADRAO_CN_SAEB':'erro_padrao_saeb'}

rename19 = {'ID_SAEB':'ano', 'ID_REGIAO':'id_regiao', 'ID_UF':'id_uf', 'ID_MUNICIPIO':'id_municipio', 'ID_AREA':'area', 'ID_ESCOLA':'id_escola',
            'ID_DEPENDENCIA_ADM':'rede', 'ID_LOCALIZACAO':'localizacao', 'ID_TURMA':'id_turma', 'ID_TURNO':'turno', 'ID_SERIE':'serie',
            'ID_ALUNO':'id_aluno', 'IN_SITUACAO_CENSO':'situacao_censo', 'IN_AMOSTRA':'amostra', 'ESTRATO':'estrato',
            'ESTRATO_CIENCIAS':'estrato_ciencias', 'IN_PREENCHIMENTO_QUESTIONARIO':'preenchimento_questionario', 'TX_RESP_Q002':'raca_cor',
            'TX_RESP_Q003A':'mora_mae', 'TX_RESP_Q003B':'mora_pai', 'TX_RESP_Q003C':'mora_irmaos', 'TX_RESP_Q003D':'mora_avos', 
            'TX_RESP_Q003E':'mora_outros_parentes', 'TX_RESP_Q001':'idioma_domicilio', 'TX_RESP_Q008A':'possui_moradia_rua_urbanizada', 
            'TX_RESP_Q008B':'possui_agua_encanada', 'TX_RESP_Q008C':'possui_eletrecidade', 'TX_RESP_Q009A':'possui_geladeira',
            'TX_RESP_Q010I':'possui_freezer', 'TX_RESP_Q010F':'possui_microondas', 'TX_RESP_Q010H':'possui_maquina_lavar_roupa',
            'TX_RESP_Q010G':'possui_aspirador_po', 'TX_RESP_Q009E':'possui_tv', 'TX_RESP_Q010A':'possui_tv_assinatura', 'TX_RESP_Q010B':'possui_internet',
            'TX_RESP_Q009C':'possui_computador', 'TX_RESP_Q009B':'possui_tablet', 'TX_RESP_Q010D':'possui_escrivaninha', 'TX_RESP_Q007':'possui_diarista',
            'TX_RESP_Q009D':'possui_casa_dormitorio', 'TX_RESP_Q010C':'possui_casa_quarto_individual', 'TX_RESP_Q009F':'possui_casa_banheiro',
            'TX_RESP_Q009G':'possui_automovel', 'TX_RESP_Q010E':'possui_casa_garagem', 'TX_RESP_Q004':'escolaridade_mae', 'TX_RESP_Q005':'escolaridade_pai',
            'TX_RESP_Q006A':'responsaveis_conversam_escola', 'TX_RESP_Q006B':'responsaveis_incentivam_estudos', 
            'TX_RESP_Q006C':'responsaveis_incentivam_realizacao_licao_casa', 'TX_RESP_Q006D':'responsaveis_incentivam_comparecer_aulas',
            'TX_RESP_Q006E':'responsaveis_comparecem_reuniao_pais', 'TX_RESP_Q011':'tempo_chegada_escola', 'TX_RESP_Q012':'forma_chegada_escola',
            'TX_RESP_Q013':'idade_entrada_escola', 'TX_RESP_Q014':'rede_ef', 'TX_RESP_Q015':'reprovacao', 'TX_RESP_Q016':'evasao_escolar_ate_final_ano',
            'TX_RESP_Q017A':'tempo_lazer', 'TX_RESP_Q017B':'tempo_cursos', 'TX_RESP_Q017C':'tempo_trabalho_domestico', 'TX_RESP_Q017D':'tempo_estudos',
            'TX_RESP_Q017E':'tempo_trabalho', 'TX_RESP_Q018A':'leitura_noticias', 'TX_RESP_Q018B':'leitura_livros_geral',
            'TX_RESP_Q018C':'leitura_historia_quadrinhos', 'TX_RESP_Q019':'pretensao_futura'}

for a in ano:
  df_mt = pd.DataFrame()
  df_lp = pd.DataFrame()
  df_ch = pd.DataFrame()
  df_cn = pd.DataFrame()
  for chunk in pd.read_csv('/content/gdrive/MyDrive/br_inep_saeb/input/saeb19_aluno_9ano.csv', dtype='string', sep=',', chunksize = 500000):
      chunk.rename(columns=rename19, inplace=True)
      chunk = pd.merge(chunk, id_uf, how='left', left_on=['id_uf'], right_on=['id_uf'])

      chunk_mt = chunk.drop(dropmt, axis=1)
      chunk_mt.rename(columns=renamemt, inplace=True)
      chunk_mt['proficiencia_saeb'] = chunk_mt['proficiencia_saeb'].astype('float64')
      condicaomt = [
                ((chunk_mt['proficiencia_saeb'] < 225)), 
                ((chunk_mt['proficiencia_saeb'] >= 225) & (chunk_mt['proficiencia_saeb'] < 300)),
                ((chunk_mt['proficiencia_saeb'] >= 300) & (chunk_mt['proficiencia_saeb'] < 350)),
                ((chunk_mt['proficiencia_saeb'] >= 350))]
      valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
      chunk_mt['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
      chunk_mt['desempenho_aluno'] = np.select(condicaomt, valores, default=chunk_mt['desempenho_aluno'])
      chunk_mt['disciplina'] = 'MT'
      df_mt = df_mt.append(chunk_mt)
      for uf in ufs:
        print("Particionando {} de 2019".format(uf))
        df_uf = df_mt[df_mt['sigla_uf'] == uf]
        df_uf = pd.concat([df_uf, add_df])
        df_uf = df_uf[ordem]
        df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
        exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano={}/sigla_uf={}/microdados_mt.csv', index=False, encoding='utf-8', na_rep='')".format(a, uf))
      
      chunk_lp = chunk.drop(droplp, axis=1)
      chunk_lp.rename(columns=renamelp, inplace=True)
      chunk_lp['proficiencia_saeb'] = chunk_lp['proficiencia_saeb'].astype('float64')
      condicaolp = [
                ((chunk_lp['proficiencia_saeb'] < 200)), 
                ((chunk_lp['proficiencia_saeb'] >= 200) & (chunk_lp['proficiencia_saeb'] < 275)),
                ((chunk_lp['proficiencia_saeb'] >= 275) & (chunk_lp['proficiencia_saeb'] < 325)),
                ((chunk_lp['proficiencia_saeb'] >= 325))]
      valores = ['Insuficiente', 'Básico', 'Proficiente', 'Avançado']
      chunk_lp['desempenho_aluno'] = '' #cria variavel classificatoria de proficiencia
      chunk_lp['desempenho_aluno'] = np.select(condicaolp, valores, default=chunk_lp['desempenho_aluno'])
      chunk_lp['disciplina'] = 'LP'
      df_lp = df_lp.append(chunk_lp)
      for uf in ufs:
        print("Particionando {} de 2019".format(uf))
        df_uf = df_lp[df_lp['sigla_uf'] == uf]
        df_uf = pd.concat([df_uf, add_df])
        df_uf = df_uf[ordem]
        df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
        exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano={}/sigla_uf={}/microdados_lp.csv', index=False, encoding='utf-8', na_rep='')".format(a, uf))
      #del df_lp   

      chunk_ch = chunk.drop(dropch, axis=1)
      chunk_ch.rename(columns=renamech, inplace=True)
      df_ch = df_ch.append(chunk_ch)
      for uf in ufs:
        print("Particionando {} de 2019".format(uf))
        df_uf = df_ch[df_ch['sigla_uf'] == uf]
        df_uf = pd.concat([df_uf, add_df])
        df_uf = df_uf[ordem]
        df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
        exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano={}/sigla_uf={}/microdados_ch.csv', index=False, encoding='utf-8', na_rep='')".format(a, uf))
      #del df_ch

      chunk_cn = chunk.drop(dropcn, axis=1)
      chunk_cn.rename(columns=renamecn, inplace=True)     
      df_cn = df_cn.append(chunk_cn)
      for uf in ufs:
        print("Particionando {} de 2019".format(uf))
        df_uf = df_cn[df_cn['sigla_uf'] == uf]
        df_uf = pd.concat([df_uf, add_df])
        df_uf = df_uf[ordem]
        df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
        exec("df_uf.to_csv('/content/gdrive/MyDrive/br_inep_saeb/output/aluno_ef_9ano/ano={}/sigla_uf={}/microdados_cn.csv', index=False, encoding='utf-8', na_rep='')".format(a, uf))
      #del df_cn

Particionando RO de 2019
Particionando AC de 2019
Particionando AM de 2019
Particionando RR de 2019
Particionando PA de 2019
Particionando AP de 2019
Particionando TO de 2019


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Particionando MA de 2019
Particionando PI de 2019
Particionando CE de 2019
Particionando RN de 2019
Particionando PB de 2019
Particionando PE de 2019
Particionando AL de 2019
Particionando SE de 2019
Particionando BA de 2019
Particionando MG de 2019
Particionando ES de 2019
Particionando RJ de 2019
Particionando SP de 2019
Particionando PR de 2019
Particionando SC de 2019
Particionando RS de 2019
Particionando MS de 2019
Particionando MT de 2019
Particionando GO de 2019
Particionando DF de 2019
Particionando RO de 2019
Particionando AC de 2019
Particionando AM de 2019
Particionando RR de 2019
Particionando PA de 2019
Particionando AP de 2019
Particionando TO de 2019
Particionando MA de 2019
Particionando PI de 2019
Particionando CE de 2019
Particionando RN de 2019
Particionando PB de 2019
Particionando PE de 2019
Particionando AL de 2019
Particionando SE de 2019
Particionando BA de 2019
Particionando MG de 2019
Particionando ES de 2019
Particionando RJ de 2019
Particionando SP de 2019
